In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

df_nhl_init = pd.read_csv('2021-2022_player_stats_naturalstattrick.csv')
df_ahl_init = pd.read_csv('2021-2022_AHL_all_players.csv')
df_name_update = pd.read_csv('2021-2022_name_updates.csv')

df_goals = pd.read_csv('2021-2022_dobber_goals.csv')
df_fg_init = pd.read_csv('2021-2022_dobber_fights.csv')
df_st_init = pd.read_csv('2021-2022_NHLcom_Player_Career.csv',encoding='latin1')
df_ass_init = pd.read_csv('2021-2022_dobber_assists.csv',encoding='latin1')
df_ca = pd.read_csv('2021-2022-EH_RAPM-EV.csv',encoding='latin1')
df_oz_init = pd.read_csv('2021-2022_NHLcom_Puck_OZ.csv',encoding='latin1')
df_pk_init = pd.read_csv('2021-2022_NHLcom_PK.csv',encoding='utf-8-sig')
df_ps_init = pd.read_csv('2021-2022_NHLcom_Shootouts.csv',encoding='utf-8-sig')
df_ex_init = pd.read_csv('2021-2022_NHLcom_Player_Career.csv',encoding='utf-8 sig')
df_ld_init = pd.read_csv('2021-2022-Elite_Prospects-Captaincy.csv', encoding = 'latin1')
df_pres = pd.read_csv('2021-2022_Sznajder_pressures.csv')
df_def = pd.read_csv('2021-2022_Sznajder_entry_defence.csv')
df_pos_ex = pd.read_csv('2021-2022_Sznajder_possession_exits.csv')
df_pos_ent = pd.read_csv('2021-2022_Sznajder_possession_carries.csv')

# VHL Player Ratings Model

Global Variables

In [2]:
games_in_season = 82
min_rate = 54
max_rate = 99
max_rate2 = 89
max_rate_f_df = 83
min_rate_d_df = 65
rate_range = max_rate - min_rate
rate_range2 = max_rate2 = min_rate
rate_range_f_df = max_rate_f_df - min_rate
rate_range_d_df = max_rate - min_rate_d_df

#### Clean datasets so players have the same names in every df

In [3]:
nhl = df_name_update['NHL Name'].to_list()
ahl = df_name_update['AHL Name'].to_list()
evolve = df_name_update['Evolving Hockey Name'].to_list()

df_ahl_init['Player'] = df_ahl_init['Player'].replace(ahl,nhl)
df_ca['Player'] = df_ca['Player'].replace(evolve,nhl)

### Players with more games in NHL than AHL

In [4]:
df_ahl_init_cl = df_ahl_init.loc[:,['Player','GP']]
df_nhl_init_cl = df_nhl_init.loc[:,['Player','GP']]

df_ahl_init_cl['League'] = 'AHL'
df_nhl_init_cl['League'] = 'NHL'

df_names = pd.concat([df_nhl_init_cl,df_ahl_init_cl]).reset_index(drop=True)

df_names = df_names.iloc[df_names.groupby('Player')['GP'].idxmax()]
df_names['Player'] = df_names['Player'].str.lower()

df_nhl_names = df_names[df_names['League'] == 'NHL'].values.tolist()

names_list = [name[0] for name in df_nhl_names]
names_list = [name.lower() for name in names_list]
df_nhl_init['Player'] = df_nhl_init['Player'].str.lower()

file_clean = df_nhl_init[df_nhl_init['Player'].isin(names_list)]

file_clean

,Unnamed: 0,Player,Team,Position,GP,TOI,Goals,Total Assists,First Assists,Second Assists,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %
0,516,connor mcdavid,EDM,C,80,1765.300000,44,79,49,30,...,1,50,68,73,75,124,26,562,484,53.73
1,230,johnny gaudreau,CGY,L,82,1522.983333,40,75,52,23,...,1,24,81,50,10,39,9,1,0,100
2,261,jonathan huberdeau,FLA,L,80,1553.650000,30,85,53,32,...,2,27,85,63,99,86,28,18,36,33.33
3,442,leon draisaitl,EDM,C,80,1787.933333,55,55,33,22,...,0,28,88,74,53,140,25,887,774,53.4
4,587,kirill kaprizov,MIN,L,81,1547.616667,47,61,39,22,...,0,43,72,57,73,131,29,5,4,55.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,639,olli juolevi,"DET, FLA",D,18,229.566667,0,0,0,0,...,0,1,7,2,14,35,10,0,0,-
934,647,max jones,ANA,L,2,19.800000,0,0,0,0,...,1,3,2,0,2,1,0,1,1,50
946,726,conor timmins,ARI,D,6,106.483333,0,0,0,0,...,0,0,2,3,7,8,9,0,0,-
974,883,egor sokolov,OTT,L,8,82.966667,0,0,0,0,...,0,0,0,1,13,11,4,0,0,-


## Games Played Multiplier

In [5]:
df_gm_mult = file_clean.loc[:,['Player','GP']]
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] <= (games_in_season * 0.5)) & (df_gm_mult['GP'] >= (games_in_season * 0.35))),-1,0)
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] < (games_in_season * 0.35)) & (df_gm_mult['GP'] >= (games_in_season * 0.2))),-2,df_gm_mult['add'])
df_gm_mult['add'] = np.where(((df_gm_mult['GP'] < (games_in_season * 0.2)) & (df_gm_mult['GP'] >= (games_in_season * 0.1))),-3,df_gm_mult['add'])
df_gm_mult['add'] = np.where((df_gm_mult['GP'] < (games_in_season * 0.1)),-4,df_gm_mult['add'])

df_gm_mult

,Player,GP,add
0,connor mcdavid,80,0
1,johnny gaudreau,82,0
2,jonathan huberdeau,80,0
3,leon draisaitl,80,0
4,kirill kaprizov,81,0
...,...,...,...
931,olli juolevi,18,-2
934,max jones,2,-4
946,conor timmins,6,-4
974,egor sokolov,8,-4


# Scoring Rating (SC)

1. (EVG+SHG)/GP (70%)
2. PPG/GP (30%)

- Clean dataframe and only return pertinent columns
- Calculate Goals/GP, Points/GP and ixG/GP

In [6]:
#Clean goals file for only NHL players
df_goals['Name'] = df_goals['Name'].str.lower()
df_goals2 = df_goals[df_goals['Name'].isin(names_list)]
df_goals2

,Name,Compare,Pos,Age,Team,GP,EVG,PPG,SHG,GWG,SOG,S%,ENG,G
0,auston matthews,NaN,C,24,TOR,73,46,16,0,10,348,17.2,4,60
1,leon draisaitl,NaN,C,26,EDM,80,31,24,1,11,278,19.8,3,55
2,chris kreider,NaN,L,31,NYR,81,24,26,3,11,258,20.2,1,52
3,alex ovechkin,NaN,L,36,WSH,77,36,16,1,5,334,15.0,9,50
4,kyle connor,NaN,L,25,WPG,79,37,8,3,9,317,14.8,6,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,markus nutivaara,NaN,D,27,FLA,1,0,0,0,0,0,NaN,0,0
992,christian jaros,NaN,D,26,NaN,11,0,0,0,0,9,0.0,0,0
998,gavin bayreuther,NaN,D,27,CBJ,43,0,0,0,0,44,0.0,0,0
1000,cam dineen,NaN,D,23,ARI,34,0,0,0,0,52,0.0,0,0


In [7]:
df_goals2 = df_goals2.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G']]
df_goals2['EVGSHG'] = df_goals2['EVG'] + df_goals['SHG']
df_goals2['EVG/GP'] = df_goals2['EVGSHG']/df_goals2['GP']
df_goals2['PPG/GP'] = df_goals2['PPG']/df_goals2['GP']

Segment players who have played more than and less than 5 games

In [8]:
df_goalsabove5 = df_goals2.loc[df_goals2['GP'] > 5]
df_goalsbelow5 = df_goals2.loc[df_goals2['GP'] <= 5]

Calculate EVG/GP, PPG/GP, SHG/GP z-score, z-score min, max, range and increment for players in > 5 games bucket

In [9]:
df_3 = df_goalsabove5.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G','EVG/GP','PPG/GP']]
df_3['even_zscore'] = zscore(df_3['EVG/GP'])
df_3['power_zscore'] = zscore(df_3['PPG/GP'])

z_max_even = df_3['even_zscore'].max()
z_min_even = df_3['even_zscore'].min()
z_max_power = df_3['power_zscore'].max()
z_min_power = df_3['power_zscore'].min()

z_range_even = z_max_even-z_min_even
z_range_power = z_max_power-z_min_power

z_incr_even = z_range_even/rate_range
z_incr_power = z_range_power/rate_range

## Goals/GP

Calculate Rating based on Goals/GP z-score for players in > 5 game bucket

In [10]:
df_4 = df_3.loc[:,['Name','Pos','GP','EVG','PPG','SHG','G','even_zscore','power_zscore','EVG/GP','PPG/GP']]

df_4['EVG_Rate'] = 102-((z_max_even-df_4['even_zscore'])/z_incr_even)
df_4['PPG_Rate'] = 102-((z_max_power-df_4['power_zscore'])/z_incr_power)

df_4['SC'] = (df_4['EVG_Rate'] * 0.7) + (df_4['PPG_Rate'] * 0.3)

Calculate Rating for players in <- 5 games bucket by adding +1 rating to base 57 rating for every goal scored

In [11]:
df_5 = df_goalsbelow5.loc[:,['Name','G']]
df_5['SC'] = 57+df_5['G']
df_goals_per_game = pd.concat([df_4,df_5], ignore_index=True)
df_goals_per_game.rename(columns={'Name':'Player'},inplace=True)
df_goals_per_game['Player'] = df_goals_per_game['Player'].str.lower()

df_sc_rate = df_goals_per_game.merge(df_gm_mult, how='left', on='Player')
df_sc_rate['SC'] = df_sc_rate['SC'] + df_sc_rate['add']
df_sc_rate = df_sc_rate.loc[:,['Player','SC']]

df_sc_rate.sort_values(by='SC',ascending=False, inplace=True)
df_sc_rate = df_sc_rate.reset_index(drop=True)
df_sc_rate.head(15)

,Player,SC
0,auston matthews,97.718124
1,alex ovechkin,89.760012
2,leon draisaitl,89.612960
3,chris kreider,87.163043
4,filip forsberg,87.003163
5,aleksander barkov,86.915914
6,kyle connor,86.569959
7,gabriel landeskog,86.296921
8,kirill kaprizov,85.869472
9,connor mcdavid,85.377184


# Checking (CK)

1. Hits/GP (60%)
2. Defender = Entry Defense Game Score (40%)
2. Forward = Forecheck Pressures/60 (40%)

In [12]:
df_ck = file_clean.loc[:,['Player','Position','GP','TOI','Hits']]
df_ck['Hits/GP'] = df_ck['Hits']/df_ck['GP']

Segment players who have played more than and less than 50min

In [13]:
df_ck_above5 = df_ck.loc[df_ck['GP'] > 5]
df_ck_below5 = df_ck.loc[df_ck['GP'] <= 5]

In [14]:
df_ck2 = df_ck_above5.loc[:,['Player','Position','GP','TOI','Hits','Hits/GP']]
df_ck3 = df_ck_below5.loc[:,['Player','Position','GP','TOI','Hits','Hits/GP']]
df_ck2['hits_zscore'] = zscore(df_ck2['Hits/GP'])
df_ck3['hits_zscore'] = zscore(df_ck3['Hits/GP'])
z_max_hits = df_ck2['hits_zscore'].max()
z_min_hits = df_ck2['hits_zscore'].min()
z_max_hits2 = df_ck3['hits_zscore'].max()
z_min_hits2 = df_ck3['hits_zscore'].min()
z_range_hits = z_max_hits-z_min_hits
z_range_hits2 = z_max_hits2-z_min_hits2
z_incr_hits = z_range_hits/rate_range
z_incr_hits2 = z_range_hits2/(rate_range-20)

In [15]:
df_ck2['Rating1'] = 100-((z_max_hits-df_ck2['hits_zscore'])/z_incr_hits)
df_ck3['Rating1'] = 79-((z_max_hits2-df_ck3['hits_zscore'])/z_incr_hits2)
df_ck4 = pd.concat([df_ck2,df_ck3])

Calculate Forwards Forecheck Recoveries/60 

In [16]:
df_pres['Name'] = df_pres['Name'].str.lower().str.replace(u'\xa0', u' ')
df_pres = df_pres[df_pres['Name'].isin(names_list)]
df_pres

,Name,TOI,Pressures/60
2,adam brooks,43.283333,8.317289
3,adam erne,208.700000,3.162434
4,adam gaudette,118.966667,5.547772
5,adam henrique,172.166667,4.878993
6,adam lowry,228.700000,7.083516
...,...,...,...
608,zach parise,230.483333,6.508063
609,zach sanford,194.733333,4.005478
610,zack kassian,237.316667,4.298055
611,zack macewen,240.766667,6.479302


In [17]:
df_pres_above40 = df_pres.loc[df_pres['TOI'] > 40]
df_pres_below40 = df_pres.loc[df_pres['TOI'] <= 40]

In [18]:
df_pres_above40['pres_zscore'] = zscore(df_pres_above40['Pressures/60'])
df_pres_below40['pres_zscore'] = zscore(df_pres_below40['Pressures/60'])
z_max_pres = df_pres_above40['pres_zscore'].max()
z_min_pres = df_pres_above40['pres_zscore'].min()
z_max_pres2 = df_pres_below40['pres_zscore'].max()
z_min_pres2 = df_pres_below40['pres_zscore'].min()
z_range_pres = z_max_pres-z_min_pres
z_range_pres2 = z_max_pres2-z_min_pres2
z_incr_pres = z_range_pres/rate_range
z_incr_pres2 = z_range_pres2/(rate_range-20)

<ipython-input-18-5f3b510f6bff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pres_above40['pres_zscore'] = zscore(df_pres_above40['Pressures/60'])
<ipython-input-18-5f3b510f6bff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pres_below40['pres_zscore'] = zscore(df_pres_below40['Pressures/60'])


In [19]:
df_pres_above40['Rating2'] = 102-((z_max_pres-df_pres_above40['pres_zscore'])/z_incr_pres)
df_pres_below40['Rating2'] = 79-((z_max_pres2-df_pres_below40['pres_zscore'])/z_incr_pres2)
df_pres_final = pd.concat([df_pres_above40,df_pres_below40])
df_pres_final.rename(columns={'Name' : 'Player'}, inplace=True)

display(df_pres_final.sort_values(by='Rating2',ascending=False))

<ipython-input-19-93013829a2e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pres_above40['Rating2'] = 102-((z_max_pres-df_pres_above40['pres_zscore'])/z_incr_pres)
<ipython-input-19-93013829a2e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pres_below40['Rating2'] = 79-((z_max_pres2-df_pres_below40['pres_zscore'])/z_incr_pres2)


,Player,TOI,Pressures/60,pres_zscore,Rating2
206,gerry mayhew,88.983333,9.439970,3.091938,102.000000
447,nino niederreiter,349.900000,9.259789,2.976533,101.065913
503,ross johnston,51.983333,9.233729,2.959841,100.930816
361,marian studenic,105.050000,9.138505,2.898851,100.437165
151,darren helm,250.650000,9.096349,2.871850,100.218622
...,...,...,...,...,...
168,drake caggiula,78.983333,0.759654,-2.467779,57.000000
331,kurtis macdermid,3.266667,0.000000,-1.348231,54.000000
362,mark friedman,5.133333,0.000000,-1.348231,54.000000
298,josh archibald,12.183333,0.000000,-1.348231,54.000000


Calculate Defenders Entry Defense Game Score

In [20]:
df_def['Player'] = df_def['Player'].str.lower().str.replace(u'\xa0', u' ')
df_def_pos = df_def.merge(file_clean, on='Player', how='left')
df_def_pos = df_def_pos[df_def_pos['Player'].isin(names_list)]
df_def_pos = df_def_pos[df_def_pos['Position'] == 'D']
df_def_pos = df_def_pos[['Player','Defense','TOI']]
df_def_pos['Defense'] = np.where(df_def_pos['Defense'] >= 3, 3, df_def_pos['Defense'])
df_def_pos

,Player,Defense,TOI
0,mackenzie weegar,3.000,1869.816667
1,noah hanifin,2.615,1725.900000
2,charlie mcavoy,2.545,1923.483333
3,miro heiskanen,2.540,1742.100000
4,brett pesce,2.460,1519.050000
...,...,...,...
929,alex vlasic,-0.375,214.633333
930,jake bean,-0.400,1378.383333
931,seth jones,-0.565,2044.366667
932,j.j. moser,-0.600,803.800000


In [21]:
df_def_above40 = df_def_pos.loc[df_def_pos['TOI'] > 40]
df_def_below40 = df_def_pos.loc[df_def_pos['TOI'] <= 40]

In [22]:
df_def_above40['def_zscore'] = zscore(df_def_above40['Defense'])
df_def_below40['def_zscore'] = zscore(df_def_below40['Defense'])
z_max_def = df_def_above40['def_zscore'].max()
z_min_def = df_def_above40['def_zscore'].min()
z_max_def2 = df_def_below40['def_zscore'].max()
z_min_def2 = df_def_below40['def_zscore'].min()
z_range_def = z_max_def-z_min_def
z_range_def2 = z_max_def2-z_min_def2
z_incr_def = z_range_def/rate_range
z_incr_def2 = z_range_def2/(rate_range-20)

<ipython-input-22-883667cab41d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def_above40['def_zscore'] = zscore(df_def_above40['Defense'])
<ipython-input-22-883667cab41d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def_below40['def_zscore'] = zscore(df_def_below40['Defense'])


In [23]:
df_def_above40['Rating2'] = 102-((z_max_def-df_def_above40['def_zscore'])/z_incr_def)
df_def_below40['Rating2'] = 79-((z_max_def2-df_def_below40['def_zscore'])/z_incr_def2)
df_def_final = pd.concat([df_def_above40,df_def_below40])

display(df_def_final.sort_values(by='Rating2',ascending=False))

<ipython-input-23-e62926efe4b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def_above40['Rating2'] = 102-((z_max_def-df_def_above40['def_zscore'])/z_incr_def)
<ipython-input-23-e62926efe4b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def_below40['Rating2'] = 79-((z_max_def2-df_def_below40['def_zscore'])/z_incr_def2)


,Player,Defense,TOI,def_zscore,Rating2
0,mackenzie weegar,3.000,1869.816667,3.403197,102.000000
1,noah hanifin,2.615,1725.900000,2.831392,97.200831
2,charlie mcavoy,2.545,1923.483333,2.727428,96.328255
3,miro heiskanen,2.540,1742.100000,2.720002,96.265928
4,brett pesce,2.460,1519.050000,2.601185,95.268698
...,...,...,...,...,...
930,jake bean,-0.400,1378.383333,-1.646510,59.617729
931,seth jones,-0.565,2044.366667,-1.891569,57.560942
932,j.j. moser,-0.600,803.800000,-1.943551,57.124654
933,slater koekkoek,-0.610,212.300000,-1.958404,57.000000


In [24]:
df_ck_concat = pd.concat([df_pres_final,df_def_final])

df_ck_merge = df_ck4.merge(df_ck_concat, how='left', on='Player')

df_ck_merge.drop_duplicates(inplace=True)
df_ck_merge['CK'] = (df_ck_merge['Rating1'] * 0.7) + (df_ck_merge['Rating2'] * 0.3)
df_ck_merge['CK'] = df_ck_merge['CK'].fillna(57)

df_ck_rate = df_ck_merge[['Player','Position','CK']]

df_ck_rate.sort_values(by='CK', ascending=False, inplace=True)

df_ck_rate.head(15).reset_index(drop=True)

<ipython-input-24-639a8190a91c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ck_rate.sort_values(by='CK', ascending=False, inplace=True)


,Player,Position,CK
0,radko gudas,D,94.803601
1,tanner jeannot,L,92.613465
2,luke schenn,D,90.764716
3,liam o'brien,C,90.701192
4,ryan reaves,R,90.229949
5,cal clutterbuck,R,90.156165
6,ross johnston,L,87.797254
7,brady tkachuk,L,87.736220
8,marcus foligno,L,87.508515
9,evander kane,L,86.987804


# Fighting (FG)

1. Fights/GP (100%)

In [25]:
df_fg_init['Name'] = df_fg_init['Name'].str.lower()
df_fg = df_fg_init[df_fg_init['Name'].isin(names_list)]

df_fg

,Name,Compare,Pos,Age,Team,GP,TOI,Fights/60,Fights
0,tanner jeannot,NaN,R,24,NSH,81,15:59,0.6,14
1,nicolas deslauriers,NaN,L,31,ANA,81,11:21,0.8,13
2,mark borowiecki,NaN,D,32,NSH,57,13:07,1.0,13
3,zack macewen,NaN,R,25,PHI,75,09:40,1.0,12
4,sam carrick,NaN,R,30,ANA,64,11:28,0.9,11
...,...,...,...,...,...,...,...,...,...
254,patric hornqvist,NaN,R,35,FLA,65,12:41,0.1,1
256,sam reinhart,NaN,R,26,FLA,78,17:46,0.0,1
257,dmitry kulikov,NaN,D,31,MIN,80,18:12,0.0,1
258,matt benning,NaN,D,27,NSH,65,16:20,0.1,1


Segment players into those who have played more or less than 5 games

In [26]:
df_fg_above5 = df_fg.loc[df_fg['GP'] > 5]
df_fg_below5 = df_fg.loc[df_fg['GP'] <= 5]

In [27]:
df_fg2 = df_fg_above5.loc[:,['Name','GP','Fights']]
df_fg2['Fights/GP'] = df_fg2['Fights']/df_fg2['GP']
df_fg2['fights_zscore'] = zscore(df_fg2['Fights/GP'])
z_max_fights = df_fg2['fights_zscore'].max()
z_min_fights = df_fg2['fights_zscore'].min()
z_range_fights = z_max_fights-z_min_fights
z_incr_fights = z_range_fights/(rate_range-10)
df_fg2['FG'] = 99-((z_max_fights-df_fg2['fights_zscore'])/z_incr_fights)
display(df_fg2.sort_values(by=['FG'],ascending=False))

,Name,GP,Fights,Fights/GP,fights_zscore,FG
6,liam o'brien,39,10,0.256410,4.494176,99.000000
12,jeffrey viel,34,8,0.235294,4.042406,95.973714
10,jonah gadjovich,43,10,0.232558,3.983870,95.581604
2,mark borowiecki,57,13,0.228070,3.887852,94.938406
30,mason geertsen,25,5,0.200000,3.287303,90.915493
...,...,...,...,...,...,...
224,vincent trocheck,81,1,0.012346,-0.727480,64.021577
186,adam larsson,82,1,0.012195,-0.730701,64.000000
228,alex debrincat,82,1,0.012195,-0.730701,64.000000
233,luke glendening,82,1,0.012195,-0.730701,64.000000


In [28]:
df_fg3 = df_fg_below5.loc[:,['Name','GP','Fights']]
df_fg3['FG'] = 55 + df_fg3['Fights']

In [29]:
df_fg_concat = df_fg2.append(df_fg3)

df_fg_rate = df_fg_concat[['Name','FG']]
df_fg_rate['Name'] = df_fg_rate['Name'].str.lower()
df_fg_rate.rename(columns={'Name' : 'Player'},inplace=True)

df_fg_rate.sort_values(by='FG',ascending=False, inplace=True)
df_fg_rate = df_fg_rate.reset_index(drop=True)
df_fg_rate.head(15)

<ipython-input-29-e2a7441c89ea>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg_rate['Name'] = df_fg_rate['Name'].str.lower()
C:\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-29-e2a7441c89ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg_rate.sort_values(by='FG',ascending=

,Player,FG
0,liam o'brien,99.000000
1,jeffrey viel,95.973714
2,jonah gadjovich,95.581604
3,mark borowiecki,94.938406
4,mason geertsen,90.915493
5,givani smith,90.292379
6,tanner jeannot,87.022953
7,sam carrick,86.884723
8,nicolas deslauriers,85.253616
9,brendan lemieux,85.182843


# Discipline (DI)

1. Minor Penalties Taken (85%)
2. Penalties Drawn / Total Penalties Taken (15%)

### Minor Penalties Taken/GP

In [30]:
df_di = file_clean.loc[:,['Player','GP','TOI','Minor','Total Penalties','Penalties Drawn']]
df_di['Penalties/GP'] = df_di['Minor']/df_di['GP']
df_di['Drawn/Taken'] = df_di['Penalties Drawn']/df_di['Total Penalties']
df_di['Drawn/Taken'] = df_di['Drawn/Taken'].fillna(0).replace([np.inf],3)

Segment players who have played more than and less than 100min

In [31]:
df_di_above125 = df_di.loc[df_di['GP'] > 10]
df_di_below125 = df_di.loc[df_di['GP'] <= 10]

In [32]:
df_di2 = df_di_above125.loc[:,['Player','GP','TOI','Penalties/GP','Drawn/Taken']]
df_di2['penalties_zscore'] = zscore(df_di2['Penalties/GP'])
df_di2['takendrawn_zscore'] = zscore(df_di2['Drawn/Taken'])

In [33]:
z_max_penalties = df_di2['penalties_zscore'].max()
z_min_penalties = df_di2['penalties_zscore'].min()
z_max_takendrawn = df_di2['takendrawn_zscore'].max()
z_min_takendrawn = df_di2['takendrawn_zscore'].min()
z_range_penalties = z_max_penalties-z_min_penalties
z_range_takendrawn = z_max_takendrawn-z_min_takendrawn
z_incr_penalties = z_range_penalties/rate_range
z_incr_takendrawn = z_range_takendrawn/rate_range

In [34]:
df_di2['Rating1'] = 57+((z_max_penalties-df_di2['penalties_zscore'])/z_incr_penalties)
df_di2['Rating2'] = 99-((z_max_takendrawn-df_di2['takendrawn_zscore'])/z_incr_takendrawn)
df_di2['DI'] = (df_di2['Rating1'] * 0.85) + (df_di2['Rating2'] * 0.15)
display(df_di2.sort_values(by=['DI'],ascending=False))

,Player,GP,TOI,Penalties/GP,Drawn/Taken,penalties_zscore,takendrawn_zscore,Rating1,Rating2,DI
457,riley sheahan,69,831.033333,0.014493,11.000000,-1.739437,11.573431,100.992095,99.000000,100.693281
398,ryan pulock,56,1183.583333,0.017857,5.000000,-1.703825,4.539123,100.758117,74.454545,96.812581
102,jack hughes,49,958.700000,0.000000,3.000000,-1.892842,2.194354,102.000000,66.272727,96.640909
652,cole perfetti,18,254.666667,0.000000,3.000000,-1.892842,2.194354,102.000000,66.272727,96.640909
658,carter rowney,26,277.383333,0.000000,3.000000,-1.892842,2.194354,102.000000,66.272727,96.640909
...,...,...,...,...,...,...,...,...,...,...
537,niko mikkola,54,917.033333,0.462963,0.384615,3.007597,-0.871883,69.803030,55.573427,67.668590
52,andrei svechnikov,78,1346.266667,0.474359,0.736842,3.128223,-0.458938,69.010490,57.014354,67.211069
81,pierre-luc dubois,81,1532.966667,0.530864,1.111111,3.726328,-0.020150,65.080808,58.545455,64.100505
193,evander kane,43,836.366667,0.581395,0.576923,4.261197,-0.646424,61.566596,56.360140,60.785628


In [35]:
df_di3 = df_di_below125.loc[:,['Player','GP','TOI','Penalties/GP','Drawn/Taken']]
df_di3['penalties_zscore'] = zscore(df_di3['Penalties/GP'])
df_di3['takendrawn_zscore'] = zscore(df_di3['Drawn/Taken'])

In [36]:
z_max_penalties2 = df_di3['penalties_zscore'].max()
z_min_penalties2 = df_di3['penalties_zscore'].min()
z_max_takendrawn2 = df_di3['takendrawn_zscore'].max()
z_min_takendrawn2 = df_di3['takendrawn_zscore'].min()
z_range_penalties2 = z_max_penalties2-z_min_penalties2
z_range_takendrawn2 = z_max_takendrawn2-z_min_takendrawn2
z_incr_penalties2 = z_range_penalties2/rate_range
z_incr_takendrawn2 = z_range_takendrawn2/rate_range

In [37]:
df_di3['Rating1'] = 59+((z_max_penalties2-df_di3['penalties_zscore'])/z_incr_penalties2)
df_di3['Rating2'] = 99-((z_max_takendrawn2-df_di3['takendrawn_zscore'])/z_incr_takendrawn2)
df_di3['DI'] = (df_di3['Rating1'] * 0.9) + (df_di3['Rating2'] * 0.1)
df_di3['DI'] = df_di3['DI'] - 15
display(df_di3.sort_values(by=['DI'],ascending=False).head(250))

,Player,GP,TOI,Penalties/GP,Drawn/Taken,penalties_zscore,takendrawn_zscore,Rating1,Rating2,DI
603,matty beniers,10,169.400000,0.000000,3.0,-0.764090,1.679552,104.000000,99.0,88.500000
733,bobby brink,10,154.483333,0.000000,3.0,-0.764090,1.679552,104.000000,99.0,88.500000
702,thomas bordeleau,8,124.716667,0.000000,3.0,-0.764090,1.679552,104.000000,99.0,88.500000
912,chase deleo,2,16.300000,0.000000,3.0,-0.764090,1.679552,104.000000,99.0,88.500000
829,petteri lindbohm,9,126.500000,0.000000,2.0,-0.764090,0.826446,104.000000,84.0,87.000000
934,max jones,2,19.800000,0.000000,1.5,-0.764090,0.399893,104.000000,76.5,86.250000
983,artemi kniazev,1,11.983333,0.000000,0.0,-0.764090,-0.879765,104.000000,54.0,84.000000
946,conor timmins,6,106.483333,0.000000,0.0,-0.764090,-0.879765,104.000000,54.0,84.000000
767,mason mctavish,9,117.716667,0.111111,3.0,-0.242431,1.679552,99.000000,99.0,84.000000
925,brett seney,2,14.033333,0.000000,0.0,-0.764090,-0.879765,104.000000,54.0,84.000000


In [38]:
df_di_concat = df_di2.append(df_di3)
df_di_rate = df_di_concat[['Player','DI']]

df_di_rate['Player'] = df_di_rate['Player'].str.lower()

df_di_rate.sort_values(by='DI',ascending=False, inplace=True)

df_di_rate.head(15)

<ipython-input-38-1046fd65b1cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_di_rate['Player'] = df_di_rate['Player'].str.lower()
<ipython-input-38-1046fd65b1cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_di_rate.sort_values(by='DI',ascending=False, inplace=True)


,Player,DI
457,riley sheahan,100.693281
398,ryan pulock,96.812581
602,aliaksei protas,96.640909
652,cole perfetti,96.640909
658,carter rowney,96.640909
102,jack hughes,96.640909
614,will butcher,96.640909
293,marcus johansson,96.154743
12,kyle connor,95.144361
714,ryan murray,95.043243


# Strength (ST)

1. Weight/Height Ratio (100%)

In [39]:
df_st_init['Player'] = df_st_init['Player'].str.lower()
df_st = df_st_init[df_st_init['Player'].isin(names_list)]

In [40]:
df_st2 = df_st.loc[:,['Player','Height','Weight']]

Convert Height to cm

In [41]:
def parse_ht(ht):
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return round((12*ft_) + in_)

df_st2['height_in'] = df_st2['Height'].apply(lambda x:parse_ht(x))

In [42]:
df_st2['ratio_zscore'] = zscore(df_st2['Weight']/df_st2['height_in'])
z_max_st = df_st2['ratio_zscore'].max()
z_min_st = df_st2['ratio_zscore'].min()
z_range_st = z_max_st-z_min_st
z_incr_st = z_range_st/rate_range
df_st2['ST'] = 99-((z_max_st-df_st2['ratio_zscore'])/z_incr_st)
df_st_rate = df_st2[['Player','height_in','ST']]

df_st_rate['Player'] = df_st_rate['Player'].str.lower()

df_st_rate.sort_values(by='ST',ascending=False,inplace=True)

df_st_rate.head(15).reset_index(drop=True)

<ipython-input-42-c9c85d3bb0a0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_st_rate['Player'] = df_st_rate['Player'].str.lower()
<ipython-input-42-c9c85d3bb0a0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_st_rate.sort_values(by='ST',ascending=False,inplace=True)


,Player,height_in,ST
0,jamie oleksiak,79,99.000000
1,david savard,73,97.970608
2,pat maroon,75,96.491262
3,alex ovechkin,75,96.491262
4,nick ritchie,75,95.264078
5,brian boyle,78,95.004481
6,anders lee,75,94.650485
7,vladimir tarasenko,73,94.188190
8,ben chiarot,75,94.036893
9,jack johnson,73,93.557787


# Endurance (EN)

1. Average TOI (100%)

In [43]:
df_en = file_clean.loc[:,['Player','GP','TOI']]
df_en['ATOI'] = df_en['TOI']/df_en['GP']

In [44]:
df_en_above125 = df_en.loc[df_en['TOI'] > 125]
df_en_below125 = df_en.loc[df_en['TOI'] <= 125]

In [45]:
df_en2 = df_en_above125.loc[:,['Player','GP','TOI','ATOI']]
df_en2['ATOI'] = np.where(df_en2['ATOI'] < 8, 8, df_en2['ATOI'])
df_en2['atoi_zscore'] = zscore(df_en2['ATOI'])
z_max_en = df_en2['atoi_zscore'].max()
z_min_en = df_en2['atoi_zscore'].min()
z_range_en = z_max_en-z_min_en
z_incr_en = z_range_en/rate_range
df_en2['EN'] = 99-((z_max_en-df_en2['atoi_zscore'])/z_incr_en)
display(df_en2.sort_values(by=['EN'],ascending=False))

,Player,GP,TOI,ATOI,atoi_zscore,EN
125,seth jones,78,2044.366667,26.209829,2.589485,99.000000
207,thomas chabot,59,1545.333333,26.192090,2.584849,98.956164
107,brent burns,82,2143.366667,26.138618,2.570874,98.824023
53,kris letang,78,2010.633333,25.777350,2.476460,97.931262
269,drew doughty,39,1003.916667,25.741453,2.467078,97.842552
...,...,...,...,...,...,...
513,dominic toninato,77,646.433333,8.395238,-2.066231,54.976710
689,brendan perlini,23,192.566667,8.372464,-2.072183,54.920430
687,kurtis macdermid,58,429.150000,8.000000,-2.169524,54.000000
568,michael pezzetta,51,399.633333,8.000000,-2.169524,54.000000


In [46]:
df_en3 = df_en_below125.loc[:,['Player','GP','TOI','ATOI']]
df_en3['EN'] = 57 + df_en3['ATOI']
display(df_en3.sort_values(by=['EN'],ascending=False))

,Player,GP,TOI,ATOI,EN
683,ryan ellis,4,91.216667,22.804167,79.804167
797,sean walker,6,110.233333,18.372222,75.372222
946,conor timmins,6,106.483333,17.747222,74.747222
769,nick blankenburg,7,123.000000,17.571429,74.571429
702,thomas bordeleau,8,124.716667,15.589583,72.589583
902,miles wood,3,44.283333,14.761111,71.761111
767,mason mctavish,9,117.716667,13.079630,70.079630
865,shane pinto,5,64.766667,12.953333,69.953333
765,kent johnson,9,111.983333,12.442593,69.442593
863,mattias norlinder,6,73.750000,12.291667,69.291667


In [47]:
df_en_concat = df_en2.append(df_en3)
df_en_rate = df_en_concat[['Player','EN']]

df_en_rate['Player'] = df_en_rate['Player'].str.lower()

df_en_rate = df_en_rate.sort_values(by=['EN'],ascending=False)
df_en_rate.head(15).reset_index(drop=True)

<ipython-input-47-9b3ee41c0d70>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_rate['Player'] = df_en_rate['Player'].str.lower()


,Player,EN
0,seth jones,99.000000
1,thomas chabot,98.956164
2,brent burns,98.824023
3,kris letang,97.931262
4,drew doughty,97.842552
5,cale makar,97.660950
6,zach werenski,97.657135
7,roman josi,97.362742
8,devon toews,96.945713
9,quinn hughes,96.574426


# Durability (DU)

1. Games Played last 3 seasons vs. 82 game season (Last Season = 60%, 2 Seasons Ago = 30%, 3 Seasons Ago = 10%) (90%)
2. Hits Given + Hits Taken (10%)

In [48]:
df_dudsh2 = pd.read_csv('2020-2021_player_stats_naturalstattrick.csv')
df_dudsh3 = pd.read_csv('2019-2020_player_stats_naturalstattrick.csv')

In [49]:
df_dudash1 = file_clean.loc[:,['Player','GP']]
df_dudash2 = df_dudsh2.loc[:,['Player','GP']]
df_dudash2['Player'] = df_dudash2['Player'].str.lower()
df_dudash3 = df_dudsh3.loc[:,['Player','GP']]
df_dudash3['Player'] = df_dudsh3['Player'].str.lower()

df_dudash1['GP%'] = df_dudash1['GP']/82
df_dudash2['GP%'] = df_dudash2['GP']/56
df_dudash3['GP%'] = df_dudash3['GP']/70

Calculate DU Rating for most recent season

In [50]:
df_dudash1_above5 = df_dudash1.loc[df_dudash1['GP'] >= 20]
df_dudash1_below5 = df_dudash1.loc[df_dudash1['GP'] < 20]

In [51]:
df_dudash1_above = df_dudash1_above5.loc[:,['Player','GP','GP%']]
df_dudash1_above['gp%_zscore'] = zscore(df_dudash1_above['GP%'])
z_max_dudash1 = df_dudash1_above['gp%_zscore'].max()
z_min_dudash1 = df_dudash1_above['gp%_zscore'].min()
z_range_dudash1 = z_max_dudash1-z_min_dudash1
z_incr_dudash1 = z_range_dudash1/rate_range
df_dudash1_above['DU_Dash1'] = 99-((z_max_dudash1-df_dudash1_above['gp%_zscore'])/z_incr_dudash1)
display(df_dudash1_above.sort_values(by=['DU_Dash1'],ascending=True))

,Player,GP,GP%,gp%_zscore,DU_Dash1
794,lias andersson,20,0.243902,-2.680330,54.000000
618,noel acciari,20,0.243902,-2.680330,54.000000
662,kevin labanc,21,0.256098,-2.619395,54.725806
709,justin dowling,22,0.268293,-2.558461,55.451613
706,kyle turris,23,0.280488,-2.497526,56.177419
...,...,...,...,...,...
388,milan lucic,82,1.000000,1.097609,99.000000
162,charlie coyle,82,1.000000,1.097609,99.000000
325,warren foegele,82,1.000000,1.097609,99.000000
132,moritz seider,82,1.000000,1.097609,99.000000


In [52]:
df_dudash1_below = df_dudash1_below5.loc[:,['Player','GP','GP%']]
df_dudash1_below['DU_Dash1'] = 55

In [53]:
df_dudash1_concat = df_dudash1_above.append(df_dudash1_below).fillna(0)
display(df_dudash1_concat)

,Player,GP,GP%,gp%_zscore,DU_Dash1
0,connor mcdavid,80,0.975610,0.975740,97.548387
1,johnny gaudreau,82,1.000000,1.097609,99.000000
2,jonathan huberdeau,80,0.975610,0.975740,97.548387
3,leon draisaitl,80,0.975610,0.975740,97.548387
4,kirill kaprizov,81,0.987805,1.036674,98.274194
...,...,...,...,...,...
931,olli juolevi,18,0.219512,0.000000,55.000000
934,max jones,2,0.024390,0.000000,55.000000
946,conor timmins,6,0.073171,0.000000,55.000000
974,egor sokolov,8,0.097561,0.000000,55.000000


Calculate DU Rating for two seasons ago

In [54]:
df_dudash2_above5 = df_dudash2.loc[df_dudash2['GP'] >= 10]
df_dudash2_below5 = df_dudash2.loc[df_dudash2['GP'] < 10]

In [55]:
df_dudash2_above = df_dudash2_above5.loc[:,['Player','GP','GP%']]
df_dudash2_above['gp%_zscore'] = zscore(df_dudash2_above['GP%'])
z_max_dudash2 = df_dudash2_above['gp%_zscore'].max()
z_min_dudash2 = df_dudash2_above['gp%_zscore'].min()
z_range_dudash2 = z_max_dudash2-z_min_dudash2
z_incr_dudash2 = z_range_dudash2/rate_range
df_dudash2_above['DU_Dash2'] = 99-((z_max_dudash2-df_dudash2_above['gp%_zscore'])/z_incr_dudash2)
display(df_dudash2_above.sort_values(by=['DU_Dash2'],ascending=True))

,Player,GP,GP%,gp%_zscore,DU_Dash2
706,rem pitlick,10,0.178571,-2.284528,54.000000
745,anton blidh,10,0.178571,-2.284528,54.000000
611,cole caufield,10,0.178571,-2.284528,54.000000
788,ryan mcleod,10,0.178571,-2.284528,54.000000
646,zac jones,10,0.178571,-2.284528,54.000000
...,...,...,...,...,...
161,alex wennberg,56,1.000000,1.017197,98.042553
159,paul stastny,56,1.000000,1.017197,98.042553
181,nils hoglander,56,1.000000,1.017197,98.042553
0,connor mcdavid,56,1.000000,1.017197,98.042553


In [56]:
df_dudash2_below = df_dudash2_below5.loc[:,['Player','GP','GP%']]
df_dudash2_below['DU_Dash2'] = 55
display(df_dudash2_below)

,Player,GP,GP%,DU_Dash2
547,tyler bertuzzi,9,0.160714,55
643,grigori denisenko,7,0.125000,55
645,rasmus sandin,9,0.160714,55
647,radim zohorna,8,0.142857,55
648,nathan gerbe,9,0.160714,55
...,...,...,...,...
906,cole smith,1,0.017857,55
908,jack ahcan,3,0.053571,55
909,brinson pasichnuk,4,0.071429,55
910,josh dunne,6,0.107143,55


In [57]:
df_dudash2_concat = df_dudash2_above.append(df_dudash2_below).fillna(0)
display(df_dudash2_concat)

,Player,GP,GP%,gp%_zscore,DU_Dash2
0,connor mcdavid,56,1.000000,1.017197,98.042553
1,leon draisaitl,56,1.000000,1.017197,98.042553
2,brad marchand,53,0.946429,0.801867,95.170213
3,mitchell marner,55,0.982143,0.945420,97.085106
4,patrick kane,56,1.000000,1.017197,98.042553
...,...,...,...,...,...
906,cole smith,1,0.017857,0.000000,55.000000
908,jack ahcan,3,0.053571,0.000000,55.000000
909,brinson pasichnuk,4,0.071429,0.000000,55.000000
910,josh dunne,6,0.107143,0.000000,55.000000


Calculate DU Rating for three seasons ago

In [58]:
df_dudash3_above5 = df_dudash3.loc[df_dudash3['GP'] >= 10]
df_dudash3_below5 = df_dudash3.loc[df_dudash3['GP'] < 10]

In [59]:
df_dudash3_above = df_dudash3_above5.loc[:,['Player','GP','GP%']]
df_dudash3_above['gp%_zscore'] = zscore(df_dudash3_above['GP%'])
z_max_dudash3 = df_dudash3_above['gp%_zscore'].max()
z_min_dudash3 = df_dudash3_above['gp%_zscore'].min()
z_range_dudash3 = z_max_dudash3-z_min_dudash3
z_incr_dudash3 = z_range_dudash3/rate_range
df_dudash3_above['DU_Dash3'] = 99-((z_max_dudash3-df_dudash3_above['gp%_zscore'])/z_incr_dudash3)
display(df_dudash3_above.sort_values(by=['DU_Dash3'],ascending=True))

,Player,GP,GP%,gp%_zscore,DU_Dash3
653,dakota mermis,10,0.142857,-2.284547,54.0
717,cole bardreau,10,0.142857,-2.284547,54.0
715,jacob middleton,10,0.142857,-2.284547,54.0
697,steven kampfer,10,0.142857,-2.284547,54.0
586,gabriel vilardi,10,0.142857,-2.284547,54.0
...,...,...,...,...,...
45,brayden schenn,71,1.014286,1.031340,99.0
309,vince dunn,71,1.014286,1.031340,99.0
38,david perron,71,1.014286,1.031340,99.0
83,phillip danault,71,1.014286,1.031340,99.0


In [60]:
df_dudash3_below = df_dudash3_below5.loc[:,['Player','GP','GP%']]
df_dudash3_below['DU_Dash3'] = 55
display(df_dudash3_below)

,Player,GP,GP%,DU_Dash3
618,bryan little,7,0.100000,55
636,ville heinola,8,0.114286,55
642,colin wilson,9,0.128571,55
644,cameron gaunce,3,0.042857,55
649,danny dekeyser,8,0.114286,55
...,...,...,...,...
877,tobias bjornfot,3,0.042857,55
878,alex yelesin,4,0.057143,55
879,anton wedin,4,0.057143,55
880,andrei chibisov,2,0.028571,55


In [61]:
df_dudash3_concat = df_dudash3_above.append(df_dudash3_below).fillna(0)
display(df_dudash3_concat)

,Player,GP,GP%,gp%_zscore,DU_Dash3
0,leon draisaitl,71,1.014286,1.031340,99.000000
1,connor mcdavid,64,0.914286,0.650829,93.836066
2,david pastrnak,70,1.000000,0.976981,98.262295
3,artemi panarin,69,0.985714,0.922623,97.524590
4,nathan mackinnon,69,0.985714,0.922623,97.524590
...,...,...,...,...,...
877,tobias bjornfot,3,0.042857,0.000000,55.000000
878,alex yelesin,4,0.057143,0.000000,55.000000
879,anton wedin,4,0.057143,0.000000,55.000000
880,andrei chibisov,2,0.028571,0.000000,55.000000


Merge all 3 season dataframes

In [62]:
df_du_m = pd.merge(df_dudash1_concat,df_dudash2_concat, how='outer',on=['Player'])
df_du_merged = pd.merge(df_du_m,df_dudash3_concat, how='outer',on=['Player']).fillna(55)
display(df_du_merged)

,Player,GP_x,GP%_x,gp%_zscore_x,DU_Dash1,GP_y,GP%_y,gp%_zscore_y,DU_Dash2,GP,GP%,gp%_zscore,DU_Dash3
0,connor mcdavid,80.0,0.975610,0.975740,97.548387,56.0,1.000000,1.017197,98.042553,64.0,0.914286,0.650829,93.836066
1,johnny gaudreau,82.0,1.000000,1.097609,99.000000,56.0,1.000000,1.017197,98.042553,70.0,1.000000,0.976981,98.262295
2,jonathan huberdeau,80.0,0.975610,0.975740,97.548387,55.0,0.982143,0.945420,97.085106,69.0,0.985714,0.922623,97.524590
3,leon draisaitl,80.0,0.975610,0.975740,97.548387,56.0,1.000000,1.017197,98.042553,71.0,1.014286,1.031340,99.000000
4,kirill kaprizov,81.0,0.987805,1.036674,98.274194,55.0,0.982143,0.945420,97.085106,55.0,55.000000,55.000000,55.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,max veronneau,55.0,55.000000,55.000000,55.000000,55.0,55.000000,55.000000,55.000000,4.0,0.057143,0.000000,55.000000
1133,danil yurtaykin,55.0,55.000000,55.000000,55.000000,55.0,55.000000,55.000000,55.000000,4.0,0.057143,0.000000,55.000000
1134,alex yelesin,55.0,55.000000,55.000000,55.000000,55.0,55.000000,55.000000,55.000000,4.0,0.057143,0.000000,55.000000
1135,anton wedin,55.0,55.000000,55.000000,55.000000,55.0,55.000000,55.000000,55.000000,4.0,0.057143,0.000000,55.000000


In [63]:
df_du_gp = df_du_merged.loc[:,['Player','GP_x','DU_Dash1','GP_y','DU_Dash2','GP','DU_Dash3']]
df_du_gp['DU_GP'] = (df_du_gp['DU_Dash1'] * 0.6) + (df_du_gp['DU_Dash2'] * 0.3) + (df_du_gp['DU_Dash3'] * 0.1)
display(df_du_gp.sort_values(by=['DU_GP'],ascending=False))

,Player,GP_x,DU_Dash1,GP_y,DU_Dash2,GP,DU_Dash3,DU_GP
77,nick suzuki,82.0,99.0,56.0,98.042553,71.0,99.000000,98.712766
27,elias lindholm,82.0,99.0,56.0,98.042553,70.0,98.262295,98.638995
327,david kampf,82.0,99.0,56.0,98.042553,70.0,98.262295,98.638995
1,johnny gaudreau,82.0,99.0,56.0,98.042553,70.0,98.262295,98.638995
107,brent burns,82.0,99.0,56.0,98.042553,70.0,98.262295,98.638995
...,...,...,...,...,...,...,...,...
895,daniel carr,55.0,55.0,6.0,55.000000,11.0,54.737705,54.973770
1060,cole bardreau,55.0,55.0,55.0,55.000000,10.0,54.000000,54.900000
934,dakota mermis,55.0,55.0,3.0,55.000000,10.0,54.000000,54.900000
814,zac jones,55.0,55.0,10.0,54.000000,55.0,55.000000,54.700000


Sum up Hits Given and Hits Taken per game

In [64]:
df_du_hits = file_clean.loc[:,['Player','GP','Hits','Hits Taken']]
df_du_hits['Hits_Sum'] = df_du_hits['Hits'] + df_du_hits['Hits Taken']
df_du_hits['Hits/GP']  = df_du_hits['Hits_Sum']/df_du_hits['GP']

In [65]:
df_du_hits_above5 = df_du_hits.loc[df_du_hits['GP'] >= 5]
df_du_hits_below5 = df_du_hits.loc[df_du_hits['GP'] < 5]

In [66]:
df_du_hits_above = df_du_hits_above5.loc[:,['Player','GP','Hits','Hits Taken','Hits_Sum','Hits/GP']]
df_du_hits_above['hits_zscore'] = zscore(df_du_hits_above['Hits/GP'])
z_max_du_hits = df_du_hits_above['hits_zscore'].max()
z_min_du_hits = df_du_hits_above['hits_zscore'].min()
z_range_du_hits = z_max_du_hits-z_min_du_hits
z_incr_du_hits = z_range_du_hits/rate_range
df_du_hits_above['Hits_Rating'] = 99-((z_max_du_hits-df_du_hits_above['hits_zscore'])/z_incr_du_hits)
display(df_du_hits_above.sort_values(by=['Hits_Rating'],ascending=False).head(25))

,Player,GP,Hits,Hits Taken,Hits_Sum,Hits/GP,hits_zscore,Hits_Rating
484,rasmus ristolainen,66,230,199,429,6.500000,3.833453,99.000000
479,radko gudas,77,355,140,495,6.428571,3.763695,98.481159
454,luke schenn,66,273,107,380,5.757576,3.108393,93.607194
472,martin fehervary,79,251,198,449,5.683544,3.036093,93.069446
186,tanner jeannot,81,318,134,452,5.580247,2.935212,92.319116
825,dmitrij jaskin,12,47,18,65,5.416667,2.775457,91.130906
490,cal clutterbuck,59,229,85,314,5.322034,2.683038,90.443514
576,connor clifton,60,156,162,318,5.300000,2.661519,90.283465
544,alexander romanov,79,227,189,416,5.265823,2.628141,90.035209
549,jani hakanpaa,80,248,173,421,5.262500,2.624896,90.011073


In [67]:
df_du_hits_below = df_du_hits_below5.loc[:,['Player','GP','Hits','Hits Taken','Hits_Sum','Hits/GP']]
df_du_hits_below['hits_zscore'] = zscore(df_du_hits_below['Hits/GP'])
z_max_du_hits2 = df_du_hits_below['hits_zscore'].max()
z_min_du_hits2 = df_du_hits_below['hits_zscore'].min()
z_range_du_hits2 = z_max_du_hits2-z_min_du_hits2
z_incr_du_hits2 = z_range_du_hits2/(rate_range-10)
df_du_hits_below['Hits_Rating'] = 89-((z_max_du_hits2-df_du_hits_below['hits_zscore'])/z_incr_du_hits2)
display(df_du_hits_below.sort_values(by=['Hits_Rating'],ascending=False))

,Player,GP,Hits,Hits Taken,Hits_Sum,Hits/GP,hits_zscore,Hits_Rating
878,leo komarov,1,3,1,4,4.00,1.326710,89.000000
902,miles wood,3,5,7,12,4.00,1.326710,89.000000
837,markus nutivaara,1,1,2,3,3.00,0.423418,77.333333
925,brett seney,2,3,3,6,3.00,0.423418,77.333333
912,chase deleo,2,5,0,5,2.50,-0.028228,71.500000
934,max jones,2,2,1,3,1.50,-0.931520,59.833333
683,ryan ellis,4,1,4,5,1.25,-1.157343,56.916667
983,artemi kniazev,1,1,0,1,1.00,-1.383166,54.000000


In [68]:
df_du_hits_concat = df_du_hits_above.append(df_du_hits_below)
display(df_du_hits_concat)

,Player,GP,Hits,Hits Taken,Hits_Sum,Hits/GP,hits_zscore,Hits_Rating
0,connor mcdavid,80,75,124,199,2.487500,-0.085201,69.854085
1,johnny gaudreau,82,10,39,49,0.597561,-1.930938,56.125984
2,jonathan huberdeau,80,99,86,185,2.312500,-0.256108,68.582923
3,leon draisaitl,80,53,140,193,2.412500,-0.158447,69.309301
4,kirill kaprizov,81,73,131,204,2.518519,-0.054908,70.079396
...,...,...,...,...,...,...,...,...
902,miles wood,3,5,7,12,4.000000,1.326710,89.000000
912,chase deleo,2,5,0,5,2.500000,-0.028228,71.500000
925,brett seney,2,3,3,6,3.000000,0.423418,77.333333
934,max jones,2,2,1,3,1.500000,-0.931520,59.833333


In [69]:
df_du_m_final = pd.merge(df_du_gp,df_du_hits_concat, how='left',on=['Player']).fillna(55)
df_du_final = df_du_m_final.loc[:,['Player','DU_GP','Hits_Rating']]
df_du_final['DU'] = (df_du_final['DU_GP'] * 0.9) + (df_du_final['Hits_Rating'] * 0.1)

df_du_rate = df_du_final[['Player','DU']]
df_du_rate = df_du_rate.groupby(['Player']).mean().reset_index()

df_du_rate['Player'] = df_du_rate['Player'].str.lower()

df_du_rate.sort_values(by='DU',ascending=False,inplace=True)
df_du_rate.head(15).reset_index(drop=True)

,Player,DU
0,brady tkachuk,96.503700
1,milan lucic,96.398608
2,tyler myers,95.980649
3,nick suzuki,95.685387
4,radko gudas,95.681759
5,alex killorn,95.663372
6,david kampf,95.636710
7,rasmus andersson,95.477261
8,brenden dillon,95.433058
9,tj brodie,95.335790


# Puck Handling (PH)

1. Points/GP (100%)

In [70]:
df_ph = file_clean.loc[:,['Player','GP','TOI','Total Points','iHDCF']]
df_ph['Points/GP'] = df_ph['Total Points']/df_ph['GP']

In [71]:
df_ph_above50 = df_ph.loc[df_ph['TOI'] > 200]
df_ph_below50 = df_ph.loc[df_ph['TOI'] <= 200]

In [72]:
df_ph_above50['ppg_zscore'] = zscore(df_ph_above50['Points/GP'])
df_ph_below50['ppg_zscore'] = zscore(df_ph_below50['Points/GP'])

z_max_ph_above50 = df_ph_above50['ppg_zscore'].max()
z_min_ph_above50 = df_ph_above50['ppg_zscore'].min()
z_range_ph_above50 = z_max_ph_above50-z_min_ph_above50
z_incr_ph_above50 = z_range_ph_above50/rate_range

z_max_ph_below50 = df_ph_below50['ppg_zscore'].max()
z_min_ph_below50 = df_ph_below50['ppg_zscore'].min()
z_range_ph_below50 = z_max_ph_below50-z_min_ph_below50
z_incr_ph_below50 = z_range_ph_below50/(rate_range-29)

df_ph_above50['PH'] = 99-((z_max_ph_above50-df_ph_above50['ppg_zscore'])/z_incr_ph_above50)
df_ph_below50['PH'] = 70-((z_max_ph_below50-df_ph_below50['ppg_zscore'])/z_incr_ph_below50)

df_ph_below50

<ipython-input-72-c2eb17b7e60f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ph_above50['ppg_zscore'] = zscore(df_ph_above50['Points/GP'])
<ipython-input-72-c2eb17b7e60f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ph_below50['ppg_zscore'] = zscore(df_ph_below50['Points/GP'])
<ipython-input-72-c2eb17b7e60f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,Player,GP,TOI,Total Points,iHDCF,Points/GP,ppg_zscore,PH
603,matty beniers,10,169.400000,9,7,0.900000,2.307875,65.520000
654,michael stone,11,192.033333,6,0,0.545455,1.026735,60.981818
683,ryan ellis,4,91.216667,5,0,1.250000,3.572590,70.000000
689,brendan perlini,23,192.566667,5,6,0.217391,-0.158711,56.782609
702,thomas bordeleau,8,124.716667,5,5,0.625000,1.314171,62.000000
706,kyle turris,23,198.300000,4,4,0.173913,-0.315819,56.226087
719,sammy blais,14,188.416667,4,9,0.285714,0.088172,57.657143
729,nathan smith,10,141.500000,4,11,0.400000,0.501140,59.120000
733,bobby brink,10,154.483333,4,9,0.400000,0.501140,59.120000
736,william eklund,9,128.950000,4,7,0.444444,0.661738,59.688889


In [73]:
df_ph_concat = pd.concat([df_ph_above50,df_ph_below50])
df_ph_concat = df_ph_concat[['Player','PH']]

df_ph_rate = df_ph_concat.merge(df_gm_mult,how='left',on='Player')
df_ph_rate['PH'] = df_ph_rate['PH'] + df_ph_rate['add']
df_ph_rate = df_ph_rate[['Player','PH']]

df_ph_rate.sort_values(by='PH',ascending=False,inplace=True)

df_ph_rate.head(15).reset_index(drop=True)

,Player,PH
0,connor mcdavid,99.000000
1,nikita kucherov,96.968345
2,auston matthews,96.499165
3,jonathan huberdeau,96.073171
4,johnny gaudreau,95.046996
5,leon draisaitl,94.243902
6,nathan mackinnon,93.624765
7,mitchell marner,93.430894
8,kirill kaprizov,93.024390
9,aleksander barkov,92.441937


# Faceoffs (FO)

1. FOW% (100%)
2. Sliding scale for every 200 draws taken less than top draw taker, subtract 1 additional rating, from max to 800 draws

In [74]:
df_fo = file_clean.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs %']]
df_fo = df_fo.replace('-',0)
df_fo['Faceoffs %'] = df_fo['Faceoffs %'].astype(float).astype(int)
df_fo['Faceoffs Taken'] = df_fo['Faceoffs Won'] + df_fo['Faceoffs Lost']
df_fo_above75 = df_fo.loc[df_fo['Faceoffs Taken'] >= 75]
df_fo_below75 = df_fo.loc[df_fo['Faceoffs Taken'] < 75]

In [75]:
df_fo2 = df_fo_above75.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs Taken','Faceoffs %']]
df_fo2['fo_zscore'] = zscore(df_fo2['Faceoffs %'])
z_max_fo = df_fo2['fo_zscore'].max()
z_min_fo = df_fo2['fo_zscore'].min()
z_range_fo = z_max_fo-z_min_fo
z_incr_fo = z_range_fo/rate_range
df_fo2['FO'] = 99-((z_max_fo-df_fo2['fo_zscore'])/z_incr_fo)

In [76]:
df_fo3 = df_fo_below75.loc[:,['Player','Position','GP','Faceoffs Won','Faceoffs Lost','Faceoffs Taken','Faceoffs %']]
df_fo3['FO'] = np.where(df_fo3['GP'] < 5, 50, 52 + (df_fo3['Faceoffs Won']*(df_fo3['Faceoffs %']/100)))
display(df_fo3.sort_values(by=['FO'],ascending=False))

,Player,Position,GP,Faceoffs Won,Faceoffs Lost,Faceoffs Taken,Faceoffs %,FO
629,tyler johnson,C,26,31,20,51,60,70.60
590,mathieu perreault,C,25,36,35,71,50,70.00
430,adam erne,L,79,34,31,65,52,69.68
33,william nylander,R,81,34,31,65,52,69.68
536,jansen harkins,C,77,28,16,44,63,69.64
...,...,...,...,...,...,...,...,...
912,chase deleo,C,2,0,2,2,0,50.00
925,brett seney,L,2,0,5,5,0,50.00
683,ryan ellis,D,4,0,0,0,0,50.00
934,max jones,L,2,1,1,2,50,50.00


1. If position is D, set FO rating to 40
2. Add sliding scale adjustment (O'Reilly has max of 1661 faceoffs so base off 1600)

In [77]:
df_fo_concat = pd.concat([df_fo2,df_fo3])
display(df_fo_concat.sort_values(by=['Faceoffs Taken'],ascending=False))

,Player,Position,GP,Faceoffs Won,Faceoffs Lost,Faceoffs Taken,Faceoffs %,fo_zscore,FO
3,leon draisaitl,C,80,887,774,1661,53,0.865038,88.090909
62,patrice bergeron,C,73,991,609,1600,61,2.208046,99.000000
27,elias lindholm,C,82,841,751,1592,52,0.697162,86.727273
88,ryan o'reilly,C,78,900,685,1585,56,1.368666,92.181818
58,anze kopitar,C,81,858,651,1509,56,1.368666,92.181818
...,...,...,...,...,...,...,...,...,...
441,brian dumoulin,D,76,0,0,0,0,NaN,52.000000
440,brayden mcnabb,D,69,0,0,0,0,NaN,52.000000
439,tyler myers,D,82,0,0,0,0,NaN,52.000000
438,justin braun,D,69,0,0,0,0,NaN,52.000000


In [78]:
def rounding(x, base=200):
    return int(base * round(x/base))
df_fo_concat['FO'] = np.where(df_fo_concat['Faceoffs Taken'] > 75, df_fo_concat['FO'] - ((1600 - df_fo_concat['Faceoffs Taken'].apply(lambda x: rounding(x,base=200)))/200), df_fo_concat['FO'])
df_fo_concat['FO'] = np.where(df_fo_concat['Position'] == 'D', 40, df_fo_concat['FO'])
df_fo_concat['FO'] = np.where((df_fo_concat['Position'] != 'D') & (df_fo_concat['Faceoffs Taken'] < 2), 51, df_fo_concat['FO'])
df_fo_rate = df_fo_concat[['Player','FO']]

df_fo_rate['Player'] = df_fo_rate['Player'].str.lower()

df_fo_rate.sort_values(by='FO',ascending=False,inplace=True)

df_fo_rate.head(15).reset_index(drop=True)

<ipython-input-78-0bbe53f1c9e2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fo_rate['Player'] = df_fo_rate['Player'].str.lower()
<ipython-input-78-0bbe53f1c9e2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fo_rate.sort_values(by='FO',ascending=False,inplace=True)


,Player,FO
0,patrice bergeron,99.000000
1,john tavares,96.636364
2,claude giroux,95.636364
3,jonathan toews,93.909091
4,bo horvat,92.545455
5,anze kopitar,92.181818
6,ryan o'reilly,92.181818
7,jordan staal,91.545455
8,luke glendening,90.909091
9,michael mcleod,90.545455


# Passing (PA)

1. EV + SH Primary assist (40%)
2. PP Primary assist (30%)
4. EV + SH Secondary assist (25%)
5. PP Secondary assist (5%)

In [79]:
df_ass_init['Name'] = df_ass_init['Name'].str.lower()
df_ass = df_ass_init[df_ass_init['Name'].isin(names_list)]

df_ass['Name'] = np.where('Â' in df_ass['Name'], df_ass['Name'].str[3:], df_ass['Name'])
df_ass['Name'] = df_ass['Name'].str.lower()

df_pa = df_ass.loc[:,['Name','GP','Prim.','Sec.','EV Prim.','PP Prim.','SH Prim.','EV Sec.','PP Sec.','SH Sec.']]
df_pa['EVSH1/GP'] = (df_pa['EV Prim.'] + df_pa['SH Prim.'])/df_pa['GP']
df_pa['PP1/GP'] = df_pa['PP Prim.']/df_pa['GP']
df_pa['EVSH2/GP'] = (df_pa['EV Sec.'] + df_pa['SH Sec.'])/df_pa['GP']
df_pa['PP2/GP'] = df_pa['PP Sec.']/df_pa['GP']
df_pa_above5 = df_pa.loc[df_pa['GP'] > 5]
df_pa_below5 = df_pa.loc[df_pa['GP'] <= 5]

<ipython-input-79-0861409f7393>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ass['Name'] = np.where('Â' in df_ass['Name'], df_ass['Name'].str[3:], df_ass['Name'])
<ipython-input-79-0861409f7393>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ass['Name'] = df_ass['Name'].str.lower()


In [80]:
df_pa2 = df_pa_above5.loc[:,['Name','EVSH1/GP','PP1/GP','EVSH2/GP','PP2/GP']]
df_pa2['evsh1_zscore'] = zscore(df_pa2['EVSH1/GP'])
df_pa2['pp1_zscore'] = zscore(df_pa2['PP1/GP'])
df_pa2['evsh2_zscore'] = zscore(df_pa2['EVSH2/GP'])
df_pa2['pp2_zscore'] = zscore(df_pa2['PP2/GP'])

z_max_evsh1 = df_pa2['evsh1_zscore'].max()
z_min_evsh1 = df_pa2['evsh1_zscore'].min()
z_range_evsh1 = z_max_evsh1-z_min_evsh1
z_incr_evsh1 = z_range_evsh1/rate_range

z_max_pp1 = df_pa2['pp1_zscore'].max()
z_min_pp1 = df_pa2['pp1_zscore'].min()
z_range_pp1 = z_max_pp1-z_min_pp1
z_incr_pp1 = z_range_pp1/rate_range

z_max_evsh2 = df_pa2['evsh2_zscore'].max()
z_min_evsh2 = df_pa2['evsh2_zscore'].min()
z_range_evsh2 = z_max_evsh2-z_min_evsh2
z_incr_evsh2 = z_range_evsh2/rate_range

z_max_pp2 = df_pa2['pp2_zscore'].max()
z_min_pp2 = df_pa2['pp2_zscore'].min()
z_range_pp2 = z_max_pp2-z_min_pp2
z_incr_pp2 = z_range_pp2/rate_range

df_pa2['EVSH1_Rating'] = 99-((z_max_evsh1-df_pa2['evsh1_zscore'])/z_incr_evsh1)
df_pa2['PP1_Rating'] = 99-((z_max_pp1-df_pa2['pp1_zscore'])/z_incr_pp1)
df_pa2['EVSH2_Rating'] = 99-((z_max_evsh2-df_pa2['evsh2_zscore'])/z_incr_evsh2)
df_pa2['PP2_Rating'] = 99-((z_max_pp2-df_pa2['pp2_zscore'])/z_incr_pp2)

df_pa2['PA'] = (df_pa2['EVSH1_Rating'] * 0.40) + (df_pa2['PP1_Rating'] * 0.30) + (df_pa2['EVSH2_Rating'] * 0.25) + (df_pa2['PP2_Rating'] * 0.05)
df_pa2['PA'] = df_pa2['PA'] + (99 - df_pa2['PA'].max())
display(df_pa2.sort_values(by=['PA'],ascending=False))

,Name,EVSH1/GP,PP1/GP,EVSH2/GP,PP2/GP,evsh1_zscore,pp1_zscore,evsh2_zscore,pp2_zscore,EVSH1_Rating,PP1_Rating,EVSH2_Rating,PP2_Rating,PA
0,jonathan huberdeau,0.412500,0.250000,0.237500,0.162500,3.564504,4.775008,2.398219,2.904687,89.398256,99.000000,82.215000,75.937500,99.000000
1,connor mcdavid,0.362500,0.250000,0.200000,0.175000,2.947977,4.775008,1.766983,3.178273,85.107558,99.000000,77.760000,77.625000,96.254346
3,artemi panarin,0.400000,0.200000,0.160000,0.226667,3.410372,3.688060,1.093664,4.309098,88.325581,90.000000,73.008000,84.600000,94.002305
52,nikita kucherov,0.319149,0.234043,0.170213,0.212766,2.413434,4.428110,1.265575,4.004854,81.387432,96.127660,74.221277,82.723404,93.274833
2,johnny gaudreau,0.524390,0.109756,0.158537,0.121951,4.944173,1.726251,1.069030,2.017198,99.000000,73.756098,72.834146,70.463415,92.648609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,mason geertsen,0.000000,0.000000,0.000000,0.000000,-1.521850,-0.659732,-1.599611,-0.651940,54.000000,54.000000,54.000000,54.000000,63.190073
948,blake comeau,0.000000,0.000000,0.000000,0.000000,-1.521850,-0.659732,-1.599611,-0.651940,54.000000,54.000000,54.000000,54.000000,63.190073
964,christian jaros,0.000000,0.000000,0.000000,0.000000,-1.521850,-0.659732,-1.599611,-0.651940,54.000000,54.000000,54.000000,54.000000,63.190073
985,conor timmins,0.000000,0.000000,0.000000,0.000000,-1.521850,-0.659732,-1.599611,-0.651940,54.000000,54.000000,54.000000,54.000000,63.190073


In [81]:
df_pa3 = df_pa_below5.loc[:,['Name','Prim.','Sec.']]
df_pa3['PA'] = 59 + (df_pa3['Prim.'] * 2) + (df_pa3['Sec.'] * 1)
display(df_pa3.sort_values(by=['PA'],ascending=False))                    

,Name,Prim.,Sec.,PA
667,ryan ellis,1,3,64
780,markus nutivaara,1,0,61
839,shane pinto,1,0,61
870,ben meyers,0,0,59
889,max jones,0,0,59
906,artemi kniazev,0,0,59
909,chase deleo,0,0,59
929,brett seney,0,0,59
956,leo komarov,0,0,59
982,miles wood,0,0,59


In [82]:
df_pa_concat = pd.concat([df_pa2,df_pa3])
df_pa_concat.rename(columns={'Name': 'Player'}, inplace=True)
df_pa_calc = df_pa_concat.merge(df_gm_mult, how='left', on='Player')
df_pa_calc['PA'] = df_pa_calc['PA'] + df_pa_calc['add']
df_pa_rate = df_pa_concat[['Player','PA']]

df_pa_rate.sort_values(by='PA',ascending=False,inplace=True)

df_pa_rate.head(15).reset_index(drop=True)

<ipython-input-82-4f2d59a447eb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pa_rate.sort_values(by='PA',ascending=False,inplace=True)


,Player,PA
0,jonathan huberdeau,99.000000
1,connor mcdavid,96.254346
2,artemi panarin,94.002305
3,nikita kucherov,93.274833
4,johnny gaudreau,92.648609
5,patrick kane,92.391906
6,nathan mackinnon,91.633776
7,j.t. miller,91.129578
8,mitchell marner,90.941672
9,mats zuccarello,90.610621


# Defence (DF)

1. xGA/60 (50%)
2. PK TOI/GP (30%)
3. Defensive Zone Starts (20%)

## xGA/60

In [83]:
df_ca.sort_values(by=['Player','GP'],ascending=False,inplace=True)
df_ca.drop_duplicates(subset='Player',keep='first',inplace=True)
df_ca2 = df_ca.loc[:,['Player','GP','Position','TOI','xGA/60']]
df_ca2['xGA/60'] = df_ca2['xGA/60'] * -1
df_ca2f = df_ca2.loc[df_ca2['Position'] != 'D']
df_ca2d = df_ca2.loc[df_ca2['Position'] == 'D']

z_max_caf = df_ca2f['xGA/60'].max()
z_min_caf = df_ca2f['xGA/60'].min()
z_range_caf = z_max_caf-z_min_caf
z_incr_caf = z_range_caf/rate_range_f_df

z_max_cad = df_ca2d['xGA/60'].max()
z_min_cad = df_ca2d['xGA/60'].min()
z_range_cad = z_max_cad-z_min_cad
z_incr_cad = z_range_cad/rate_range_d_df

df_ca2f['xGA60_Rating'] = 85-((z_max_caf-df_ca2f['xGA/60'])/z_incr_caf)
df_ca2d['xGA60_Rating'] = 100-((z_max_cad-df_ca2d['xGA/60'])/z_incr_cad)
df_ca_concat = pd.concat([df_ca2f,df_ca2d])
df_ca_concat['Player'] = df_ca_concat['Player'].str.lower()
display(df_ca_concat.sort_values(by=['xGA60_Rating'], ascending=False))

<ipython-input-83-5db3dc2662a1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca2f['xGA60_Rating'] = 85-((z_max_caf-df_ca2f['xGA/60'])/z_incr_caf)
<ipython-input-83-5db3dc2662a1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca2d['xGA60_Rating'] = 100-((z_max_cad-df_ca2d['xGA/60'])/z_incr_cad)


,Player,GP,Position,TOI,xGA/60,xGA60_Rating
504,jonas siegenthaler,70,D,1255.53,0.304,100.000000
502,jonas brodin,73,D,1334.20,0.264,97.926829
383,ilya lyubushkin,46,D,702.10,0.252,97.304878
200,charlie mcavoy,78,D,1443.63,0.251,97.253049
307,dylan demelo,76,D,1185.60,0.241,96.734756
...,...,...,...,...,...,...
780,nikita kucherov,47,R,728.78,-0.290,58.065152
591,kyle connor,79,L,1289.60,-0.293,57.933333
421,jake leschyshyn,41,C,426.48,-0.319,56.790909
648,mark scheifele,67,C,1133.18,-0.336,56.043939


## Defensive Zone Starts

In [84]:
df_oz_init['skaterFullName'] = df_oz_init['skaterFullName'].str.lower()
df_oz = df_oz_init[df_oz_init['skaterFullName'].isin(names_list)]

df_oz2 = df_oz.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2['zoneStartPct'] = np.where(df_oz2['zoneStartPct'] < 0.2, 0.2, df_oz2['zoneStartPct'])
df_oz2['zoneStartPct'] = np.where(df_oz2['zoneStartPct'] > 0.75, 0.75, df_oz2['zoneStartPct']) 

df_oz2f = df_oz2.loc[df_oz2['positionCode'] != 'D']
df_oz2d = df_oz2.loc[df_oz2['positionCode'] == 'D']

df_oz2f_above10 = df_oz2f.loc[df_oz2f['gamesPlayed'] >= 10]
df_oz2f_below10 = df_oz2f.loc[df_oz2f['gamesPlayed'] < 10]
df_oz2d_above10 = df_oz2d.loc[df_oz2d['gamesPlayed'] >= 10]
df_oz2d_below10 = df_oz2d.loc[df_oz2d['gamesPlayed'] < 10]

df_oz2f_a10 = df_oz2f_above10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2f_a10['zoneStartPct'] = 1 -df_oz2f_a10['zoneStartPct']
df_oz2f_a10['oz_zscore'] = zscore(df_oz2f_a10['zoneStartPct'])
z_max_oz2f_a10 = df_oz2f_a10['oz_zscore'].max()
z_min_oz2f_a10 = df_oz2f_a10['oz_zscore'].min()
z_range_oz2f_a10 = z_max_oz2f_a10-z_min_oz2f_a10
z_incr_oz2f_a10 = z_range_oz2f_a10/rate_range_f_df

df_oz2f_b10 = df_oz2f_below10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2f_b10['zoneStartPct'] = 1 - df_oz2f_b10['zoneStartPct'].fillna(0)
df_oz2f_b10['oz_zscore'] = zscore(df_oz2f_b10['zoneStartPct'])
z_max_oz2f_b10 = df_oz2f_b10['oz_zscore'].max()
z_min_oz2f_b10 = df_oz2f_b10['oz_zscore'].min()
z_range_oz2f_b10 = z_max_oz2f_b10-z_min_oz2f_b10
z_incr_oz2f_b10 = z_range_oz2f_b10/(rate_range_f_df-10)

df_oz2d_a10 = df_oz2d_above10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2d_a10['zoneStartPct'] = 1 -df_oz2d_a10['zoneStartPct']
df_oz2d_a10['oz_zscore'] = zscore(df_oz2d_a10['zoneStartPct'])
z_max_oz2d_a10 = df_oz2d_a10['oz_zscore'].max()
z_min_oz2d_a10 = df_oz2d_a10['oz_zscore'].min()
z_range_oz2d_a10 = z_max_oz2d_a10-z_min_oz2d_a10
z_incr_oz2d_a10 = z_range_oz2d_a10/rate_range_d_df

df_oz2d_b10 = df_oz2d_below10.loc[:,['skaterFullName','positionCode','gamesPlayed','zoneStartPct']]
df_oz2d_b10['zoneStartPct'] = 1 - df_oz2d_b10['zoneStartPct']
df_oz2d_b10['zoneStartPct'].fillna(0.25,inplace=True)
df_oz2d_b10['oz_zscore'] = zscore(df_oz2d_b10['zoneStartPct'])
z_max_oz2d_b10 = df_oz2d_b10['oz_zscore'].max()
z_min_oz2d_b10 = df_oz2d_b10['oz_zscore'].min()
z_range_oz2d_b10 = z_max_oz2d_b10-z_min_oz2d_b10
z_incr_oz2d_b10 = z_range_oz2d_b10/(rate_range_d_df-10)

df_oz2d_a10['OZ%_Rating'] = 100-((z_max_oz2d_a10-df_oz2d_a10['oz_zscore'])/z_incr_oz2d_a10)
df_oz2d_b10['OZ%_Rating'] = 80-((z_max_oz2d_b10-df_oz2d_b10['oz_zscore'])/z_incr_oz2d_b10)
df_oz2f_a10['OZ%_Rating'] = 83-((z_max_oz2f_a10-df_oz2f_a10['oz_zscore'])/z_incr_oz2f_a10)
df_oz2f_b10['OZ%_Rating'] = 70-((z_max_oz2f_b10-df_oz2f_b10['oz_zscore'])/z_incr_oz2f_b10)
df_oz_concat = pd.concat([df_oz2d_a10,df_oz2d_b10,df_oz2f_a10,df_oz2f_b10])
df_oz_concat.rename(columns={'skaterFullName': 'Player'}, inplace=True)
df_oz_concat['Player'] = df_oz_concat['Player'].str.lower()
display(df_oz_concat.sort_values(by=['OZ%_Rating'], ascending=False))

,Player,positionCode,gamesPlayed,zoneStartPct,oz_zscore,OZ%_Rating
372,niko mikkola,D,54,0.692,2.676104,100.000000
383,nicolas meloche,D,50,0.681,2.520597,99.153846
189,marco scandella,D,70,0.680,2.506460,99.076923
692,vladislav gavrikov,D,80,0.660,2.223721,97.538462
12,nikita zaitsev,D,62,0.659,2.209584,97.461538
...,...,...,...,...,...,...
496,evgeny kuznetsov,C,79,0.261,-2.595435,54.580000
314,alex ovechkin,L,77,0.253,-2.679619,54.158182
548,patrick kane,R,78,0.250,-2.711188,54.000000
482,hendrix lapierre,C,6,0.308,-1.607782,53.165029


## PK TOI/GP

In [85]:
df_pk_init['skaterFullName'] = df_pk_init['skaterFullName'].str.lower()
df_pk = df_pk_init[df_pk_init['skaterFullName'].isin(names_list)]

df_pkf = df_pk.loc[df_pk['positionCode'] != 'D']
df_pkd = df_pk.loc[df_pk['positionCode'] == 'D']

df_pkf_a10 = df_pkf.loc[df_pkf['gamesPlayed'] >= 10]
df_pkf_b10 = df_pkf.loc[df_pkf['gamesPlayed'] < 10]
df_pkd_a10 = df_pkd.loc[df_pkd['gamesPlayed'] >= 10]
df_pkd_b10 = df_pkd.loc[df_pkd['gamesPlayed'] < 10]

df_pkf_a102 = df_pkf_a10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkf_a102['pktoi_zscore'] = zscore(df_pkf_a102['shTimeOnIcePerGame'])
z_max_pkf_a102 = df_pkf_a102['pktoi_zscore'].max()
z_min_pkf_a102 = df_pkf_a102['pktoi_zscore'].min()
z_range_pkf_a102 = z_max_pkf_a102-z_min_pkf_a102
z_incr_pkf_a102 = z_range_pkf_a102/rate_range_f_df

df_pkf_b102 = df_pkf_b10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkf_b102['pktoi_zscore'] = zscore(df_pkf_b102['shTimeOnIcePerGame'])
z_max_pkf_b102 = df_pkf_b102['pktoi_zscore'].max()
z_min_pkf_b102 = df_pkf_b102['pktoi_zscore'].min()
z_range_pkf_b102 = z_max_pkf_b102-z_min_pkf_b102
z_incr_pkf_b102 = z_range_pkf_b102/(rate_range_f_df-10)

df_pkd_a102 = df_pkd_a10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkd_a102['pktoi_zscore'] = zscore(df_pkd_a102['shTimeOnIcePerGame'])
z_max_pkd_a102 = df_pkd_a102['pktoi_zscore'].max()
z_min_pkd_a102 = df_pkd_a102['pktoi_zscore'].min()
z_range_pkd_a102 = z_max_pkd_a102-z_min_pkd_a102
z_incr_pkd_a102 = z_range_pkd_a102/rate_range_d_df

df_pkd_b102 = df_pkd_b10.loc[:,['skaterFullName','positionCode','gamesPlayed','shTimeOnIcePerGame']]
df_pkd_b102['pktoi_zscore'] = zscore(df_pkd_b102['shTimeOnIcePerGame'])
z_max_pkd_b102 = df_pkd_b102['pktoi_zscore'].max()
z_min_pkd_b102 = df_pkd_b102['pktoi_zscore'].min()
z_range_pkd_b102 = z_max_pkd_b102-z_min_pkd_b102
z_incr_pkd_b102 = z_range_pkd_b102/(rate_range_d_df-5)

df_pkf_a102['PKTOI_Rating'] = 90-((z_max_pkf_a102-df_pkf_a102['pktoi_zscore'])/z_incr_pkf_a102)
df_pkf_b102['PKTOI_Rating'] = 70-((z_max_pkf_b102-df_pkf_b102['pktoi_zscore'])/z_incr_pkf_b102)
df_pkd_a102['PKTOI_Rating'] = 100-((z_max_pkd_a102-df_pkd_a102['pktoi_zscore'])/z_incr_pkd_a102)
df_pkd_b102['PKTOI_Rating'] = 83-((z_max_pkd_b102-df_pkd_b102['pktoi_zscore'])/z_incr_pkd_b102)
df_pktoi_concat = pd.concat([df_pkf_a102,df_pkf_b102,df_pkd_a102,df_pkd_b102])
df_pktoi_concat.rename(columns={'skaterFullName': 'Player'}, inplace=True)
df_pktoi_concat['Player'] = df_pktoi_concat['Player'].str.lower()
display(df_pktoi_concat.sort_values(by=['PKTOI_Rating'], ascending=False))

,Player,positionCode,gamesPlayed,shTimeOnIcePerGame,pktoi_zscore,PKTOI_Rating
0,mattias ekholm,D,76,195.750,1.957572,100.000000
1,ryan mcdonagh,D,71,195.070,1.945138,99.881890
2,jaccob slavin,D,79,193.670,1.919539,99.638723
3,andrew peeke,D,82,193.292,1.912627,99.573068
4,nick holden,D,76,191.947,1.888034,99.339453
...,...,...,...,...,...,...
929,brett seney,L,2,0.000,-0.685059,51.000000
962,thomas bordeleau,C,8,0.000,-0.685059,51.000000
976,chase deleo,C,2,0.000,-0.685059,51.000000
984,hendrix lapierre,C,6,0.000,-0.685059,51.000000


In [86]:
df_df = df_ca_concat.merge(df_oz_concat, how='left',on='Player').merge(df_pktoi_concat, how='left',on='Player')
df_df['xGA60_Rating'] = df_df['xGA60_Rating'].fillna(55)
df_df['OZ%_Rating'] = df_df['OZ%_Rating'].fillna(55)
df_df['PKTOI_Rating'] = df_df['PKTOI_Rating'].fillna(55)

df_df['DF'] = (df_df['xGA60_Rating'] * 0.50) + (df_df['OZ%_Rating'] * 0.30) + (df_df['PKTOI_Rating'] * 0.20)
df_df_rate = df_df.merge(df_gm_mult, how='left', on='Player')
df_df_rate['DF'] = df_df_rate['DF'] + df_df_rate['add']
df_df_rate = df_df_rate[['Player','DF']]

df_df_rate.sort_values(by='DF',ascending=False,inplace=True)
df_df_rate.head(15).reset_index(drop=True)

,Player,DF
0,jonas siegenthaler,95.553153
1,andrew peeke,93.892991
2,ryan mcdonagh,93.755201
3,connor murphy,93.423239
4,nick holden,93.214467
5,ilya lyubushkin,93.076499
6,jonas brodin,92.516100
7,matt benning,92.479967
8,dylan demelo,92.405402
9,derek forbort,92.250282


# PS

1. Season PS% (70%)
2. Career PS% (30%)

In [87]:
df_ps_init['skaterFullName'] = df_ps_init['skaterFullName'].str.lower()
df_ps = df_ps_init[df_ps_init['skaterFullName'].isin(names_list)]

df_ps_season = df_ps.loc[:,['skaterFullName','shootoutShootingPct']]
df_ps_career = df_ps.loc[:,['skaterFullName','careerShootoutShots','careerShootoutShootingPct']]

df_ps_season.fillna(0,inplace=True)
df_ps_career.fillna(0,inplace=True)

In [88]:
df_ps_season['so%season_zscore'] = zscore(df_ps_season['shootoutShootingPct'])
z_max_ps_season = df_ps_season['so%season_zscore'].max()
z_min_ps_season = df_ps_season['so%season_zscore'].min()
z_range_ps_season = z_max_ps_season-z_min_ps_season
z_incr_ps_season = z_range_ps_season/rate_range

df_ps_career['so%career_zscore'] = zscore(df_ps_career['careerShootoutShootingPct'])
z_max_ps_career = df_ps_career['so%career_zscore'].max()
z_min_ps_career = df_ps_career['so%career_zscore'].min()
z_range_ps_career = z_max_ps_career-z_min_ps_career
z_incr_ps_career = z_range_ps_career/rate_range

df_ps_season['Season'] = 99-((z_max_ps_season-df_ps_season['so%season_zscore'])/z_incr_ps_season)
df_ps_career['Career'] = 99-((z_max_ps_career-df_ps_career['so%career_zscore'])/z_incr_ps_career)

df_ps_final = df_ps_season.merge(df_ps_career, how='left',on=['skaterFullName'])
df_ps_final['PS'] = (df_ps_final['Season'] * 0.7) + (df_ps_final['Career'] * 0.3)
df_ps_final['PS'] = np.where(df_ps_final['PS'] >= 99, 85, df_ps_final['PS'])

df_ps_rate = df_ps_final[['skaterFullName','PS']]

df_ps_rate['skaterFullName'] = df_ps_rate['skaterFullName'].str.lower()

df_ps_rate.rename(columns={'skaterFullName': 'Player'}, inplace=True)

df_ps_rate.sort_values(by='PS',ascending=False,inplace=True)
display(df_ps_rate.head(15).reset_index(drop=True))


<ipython-input-88-d7e01850db8f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ps_rate['skaterFullName'] = df_ps_rate['skaterFullName'].str.lower()
C:\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-88-d7e01850db8f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ps_rate.sort_val

,Player,PS
0,lias andersson,94.499910
1,joonas donskoi,93.093750
2,jakob chychrun,92.250000
3,alexander kerfoot,92.250000
4,adam gaudette,92.250000
5,kyle turris,90.626490
6,nick bjugstad,90.264690
7,kevin hayes,89.999955
8,tyler johnson,89.999955
9,alex galchenyuk,89.795430


# EX

1. GP: Up to 1500 (100%)

In [89]:
df_ex_init['Player'] = df_ex_init['Player'].str.lower()
df_ex = df_ex_init[df_ex_init['Player'].isin(names_list)]

df_ex['GP'] = np.where(df_ex['GP'] > 1400, 1400, df_ex['GP'])
df_ex['gp_zscore'] = zscore(df_ex['GP'])
z_max_ex = df_ex['gp_zscore'].max()
z_min_ex = df_ex['gp_zscore'].min()
z_range_ex = z_max_ex-z_min_ex
z_incr_ex = z_range_ex/rate_range

df_ex['EX'] = 99-((z_max_ex-df_ex['gp_zscore'])/z_incr_ex)
df_ex['name'] = df_ex['Player'].str.lower()
df_ex['name'] = df_ex['Player'].str.replace(' ','-')
df_ex['UrlLink'] = 'https://www.nhl.com/player/' + df_ex['name'] + '-' + df_ex['Id'].astype(str)

df_ex_rate = df_ex[['Id','Player','UrlLink','Birth Date','Birth City','Birth State/Province','Birth Country','Height','Weight','Rookie','EX']]

df_ex_rate['Player'] = df_ex_rate['Player'].str.lower()

df_ex_rate.sort_values(by='EX',ascending=False,inplace=True)

df_ex_rate.head(15).reset_index(drop=True)

df_view = df_ex_rate[['Player','EX']]

df_view.head(15).reset_index(drop=True)

<ipython-input-89-887e00de8115>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex['GP'] = np.where(df_ex['GP'] > 1400, 1400, df_ex['GP'])
<ipython-input-89-887e00de8115>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex['gp_zscore'] = zscore(df_ex['GP'])
<ipython-input-89-887e00de8115>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,Player,EX
0,zdeno chara,99.000000
1,joe thornton,99.000000
2,dustin brown,95.654753
3,ryan suter,95.140100
4,alex ovechkin,94.947105
5,duncan keith,94.368120
6,brent burns,94.207291
7,jason spezza,94.110793
8,patrice bergeron,93.081487
9,anze kopitar,92.888492


# LD

1. Captaincy/Assistant (80%)
2. GP (20%)

In [90]:
df_ld_init['Player'] = df_ld_init['Player'].str.lower()
df_ld = df_ld_init[df_ld_init['Player'].isin(names_list)]

df_ld2 = df_ld[df_ld.set_index(['Player']).index.isin(df_ex.set_index(['Player']).index)]
df_ld2['score'] = (df_ld2['Captaincy'] * 3) + (df_ld2['Assistant'] * 1)
df_ld2['score'] = np.where(df_ld2['score'] > 35, 35, df_ld2['score'])

df_ld2['captass_zscore'] = zscore(df_ld2['score'])
z_max_ld = df_ld2['captass_zscore'].max()
z_min_ld = df_ld2['captass_zscore'].min()
z_range_ld = z_max_ld-z_min_ld
z_incr_ld = z_range_ld/(rate_range-10)

df_ld2['LD_Rating'] = 99-((z_max_ld-df_ld2['captass_zscore'])/z_incr_ld)
df_ld_merge = df_ex_rate.merge(df_ld2, how='left', on=['Player']).fillna(58)
df_ld_merge['LD'] = (df_ld_merge['LD_Rating'] * 0.9) + (df_ld_merge['EX'] * 0.1)

df_ld_rate = df_ld_merge[['Player','LD']]

df_ld_rate['Player'] = df_ld_rate['Player'].str.lower()

df_ld_rate.sort_values(by='LD',ascending=False,inplace=True)

df_ld_rate.head(15).reset_index(drop=True)

<ipython-input-90-beed39c099cd>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ld_rate['Player'] = df_ld_rate['Player'].str.lower()
<ipython-input-90-beed39c099cd>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ld_rate.sort_values(by='LD',ascending=False,inplace=True)


,Player,LD
0,zdeno chara,99.000000
1,alex ovechkin,98.594711
2,ryan getzlaf,98.218370
3,joe thornton,98.073529
4,sidney crosby,98.060758
5,jonathan toews,97.758399
6,gabriel landeskog,92.238269
7,dustin brown,92.180181
8,claude giroux,91.285971
9,jamie benn,91.060810


# SK

In [91]:
df_ent = df_pos_ent.loc[:,['Player','TOI','Carries/60']]
df_ent['Player'] = df_ent['Player'].str.lower()
df_ent['Player'] = df_ent['Player'].str.split().str.join(' ')
df_ent = df_ent[df_ent['Player'].isin(names_list)]

df_ext = df_pos_ex.loc[:,['Player','TOI','Possessions/60']]
df_ext['Player'] = df_ext['Player'].str.lower()
df_ext['Player'] = df_ext['Player'].str.split().str.join(' ')
df_ext = df_ext[df_ext['Player'].isin(names_list)]
df_ext['Possessions/60'] = np.where(df_ext['Possessions/60'] > 11, 11, df_ext['Possessions/60'])
df_ext['Possessions/60'] = np.where(df_ext['Possessions/60'] <= 2 , 2, df_ext['Possessions/60'])

df_ent_a = df_ent.loc[df_ent['TOI'] > 25]
df_ent_b = df_ent.loc[df_ent['TOI'] <= 25]
df_ext_a = df_ext.loc[df_ext['TOI'] > 25]
df_ext_b = df_ext.loc[df_ext['TOI'] <= 25]

df_ent_a

,Player,TOI,Carries/60
2,adam brooks,43.283333,6.931074
3,adam erne,208.700000,5.749880
4,adam gaudette,118.966667,6.556458
5,adam henrique,172.166667,9.060987
6,adam lowry,228.700000,8.132925
...,...,...,...
608,zach parise,230.483333,7.549353
609,zach sanford,194.733333,6.162273
610,zack kassian,237.316667,4.803708
611,zack macewen,240.766667,7.226914


In [92]:
df_ent_a['Carries/60'] = np.where(df_ent_a['Carries/60'] >= 21, 21, df_ent_a['Carries/60'])
df_ent_a['carries/60_zscore'] = zscore(df_ent_a['Carries/60'])
z_max_ent_a = df_ent_a['carries/60_zscore'].max()
z_min_ent_a = df_ent_a['carries/60_zscore'].min()
z_range_ent_a = z_max_ent_a-z_min_ent_a
z_incr_ent_a = z_range_ent_a/rate_range

df_ent_b['Carries/60'] = np.where(df_ent_b['Carries/60'] >= 18, 18, df_ent_b['Carries/60'])
df_ent_b['carries/60_zscore'] = zscore(df_ent_b['Carries/60'])
z_max_ent_b = df_ent_b['carries/60_zscore'].max()
z_min_ent_b = df_ent_b['carries/60_zscore'].min()
z_range_ent_b = z_max_ent_b-z_min_ent_b
z_incr_ent_b = z_range_ent_b/(rate_range-20)

df_ent_a['SK'] = 99-((z_max_ent_a-df_ent_a['carries/60_zscore'])/z_incr_ent_a)
df_ent_b['SK'] = 79-((z_max_ent_b-df_ent_b['carries/60_zscore'])/z_incr_ent_b)

df_sk_forwards = pd.concat([df_ent_a,df_ent_b])
df_sk_forwards.fillna(60, inplace=True)
display(df_sk_forwards.sort_values(by=['SK'],ascending=False))

<ipython-input-92-9bf5ea49c09e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ent_a['Carries/60'] = np.where(df_ent_a['Carries/60'] >= 21, 21, df_ent_a['Carries/60'])
<ipython-input-92-9bf5ea49c09e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ent_a['carries/60_zscore'] = zscore(df_ent_a['Carries/60'])
<ipython-input-92-9bf5ea49c09e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,Player,TOI,Carries/60,carries/60_zscore,SK
374,mathew barzal,299.016667,21.000000,3.588096,99.000000
445,nikolaj ehlers,214.766667,20.673599,3.491773,98.257572
389,matty beniers,26.983333,20.012353,3.296636,96.753509
323,kirill kaprizov,402.950000,19.952848,3.279075,96.618158
139,connor mcdavid,490.233333,18.970558,2.989196,94.383852
...,...,...,...,...,...
308,justin dowling,49.333333,1.216216,-2.250214,54.000000
96,brett seney,7.166667,0.000000,-1.385997,54.000000
298,josh archibald,12.183333,0.000000,-1.385997,54.000000
331,kurtis macdermid,3.266667,0.000000,-1.385997,54.000000


In [93]:
df_ext_a['pos/60_zscore'] = zscore(df_ext_a['Possessions/60'])
z_max_ext_a = df_ext_a['pos/60_zscore'].max()
z_min_ext_a = df_ext_a['pos/60_zscore'].min()
z_range_ext_a = z_max_ext_a-z_min_ext_a
z_incr_ext_a = z_range_ext_a/rate_range

df_ext_b['pos/60_zscore'] = zscore(df_ext_b['Possessions/60'])
z_max_ext_b = df_ext_b['pos/60_zscore'].max()
z_min_ext_b = df_ext_b['pos/60_zscore'].min()
z_range_ext_b = z_max_ext_b-z_min_ext_b
z_incr_ext_b = z_range_ext_b/(rate_range-20)

df_ext_a['SK'] = 99-((z_max_ext_a-df_ext_a['pos/60_zscore'])/z_incr_ext_a)
df_ext_b['SK'] = 79-((z_max_ext_b-df_ext_b['pos/60_zscore'])/z_incr_ext_b)

df_sk_def = pd.concat([df_ext_a,df_ext_b])

display(df_sk_def.sort_values(by=['SK'],ascending=False))

<ipython-input-93-4befce5e1348>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ext_a['pos/60_zscore'] = zscore(df_ext_a['Possessions/60'])
<ipython-input-93-4befce5e1348>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ext_b['pos/60_zscore'] = zscore(df_ext_b['Possessions/60'])
<ipython-input-93-4befce5e1348>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,Player,TOI,Possessions/60,pos/60_zscore,SK
272,sean walker,32.133333,11.000000,2.681120,99.000000
48,charlie mcavoy,432.833333,10.951097,2.656756,98.755487
304,will borgen,100.233333,10.774859,2.568951,97.874293
279,thomas chabot,325.383333,10.510680,2.437333,96.553399
217,morgan rielly,521.816667,10.233479,2.299228,95.167396
...,...,...,...,...,...
50,christian jaros,19.166667,2.000000,-0.682908,54.000000
142,joel edmundson,13.083333,2.000000,-0.682908,54.000000
195,markus nutivaara,6.850000,2.000000,-0.682908,54.000000
173,kris russell,97.433333,2.000000,-1.802813,54.000000


In [94]:
df_sk_merge = pd.concat([df_sk_forwards,df_sk_def])
df_sk_merge = df_sk_merge.groupby(['Player']).mean().reset_index()
df_sk_rate = df_sk_merge.merge(df_gm_mult, on='Player', how='left')
df_sk_rate['SK'] = df_sk_rate['SK'] + df_sk_rate['add']
df_sk_rate = df_sk_rate[['Player','SK']]

df_sk_rate.sort_values(by='SK',ascending=False,inplace=True)

df_sk_rate.head(30).reset_index(drop=True)

,Player,SK
0,mathew barzal,99.000000
1,charlie mcavoy,98.755487
2,nikolaj ehlers,98.257572
3,will borgen,96.874293
4,kirill kaprizov,96.618158
5,thomas chabot,96.553399
6,morgan rielly,95.167396
7,sean walker,95.000000
8,miro heiskanen,94.760261
9,samuel girard,94.459430


# OV

1. CK
2. FG
3. DI
4. SK
5. ST
6. EN
7. DU
8. PH
9. FO
10. PA
11. SC
12. DF
13. PS
14. EX
15. LD

In [95]:
df_ov = df_ck_rate.merge(df_fg_rate, how='left',on=['Player']).merge(df_di_rate, how='left',on=['Player']).merge(df_sk_rate,how='left',on=['Player']).merge(df_st_rate,how='left',on=['Player']).merge(df_en_rate,how='left',on=['Player']).merge(df_du_rate,how='left',on=['Player']).merge(df_ph_rate,how='left',on=['Player']).merge(df_fo_rate,how='left',on=['Player']).merge(df_pa_rate,how='left',on=['Player']).merge(df_sc_rate,how='left',on=['Player']).merge(df_df_rate,how='left',on=['Player']).merge(df_ps_rate,how='left',on=['Player']).merge(df_ex_rate,how='left',on=['Player']).merge(df_ld_rate,how='left',on=['Player'])
df_ov['Rookie'] = np.where(df_ov['Rookie'].astype(str).str.contains('True'), 1, 0)
df_ov['PO'] = 1
df_ov['Contract'] = 1
df_ov['Salary'] = 1
df_ov['Year'] = df_ov['Birth Date'].str.slice(start=0,stop=4)
df_ov['Month'] = df_ov['Birth Date'].str.slice(start=5,stop=7)
df_ov['Day'] = df_ov['Birth Date'].str.slice(start=8)

df_ov = df_ov[['Id','Player','Position','Birth Country','Rookie','Year','Month','Day','Weight','height_in','Contract','Salary','UrlLink','CK','FG','DI','SK','ST','EN','DU','PH','FO','PA','SC','DF','PS','EX','LD','PO']]

df_ov['FG'] = df_ov['FG'].fillna(40)
df_ov.fillna(60,inplace=True)

df_ov_f = df_ov[df_ov['Position'] != 'D']
df_ov_d = df_ov[df_ov['Position'] == 'D']

df_ov_d['OV'] = (df_ov_d['CK'] *0.08) + (df_ov_d['DI'] *0.02) + (df_ov_d['SK'] *0.10) + (df_ov_d['ST'] *0.05) + (df_ov_d['EN'] *0.09) + (df_ov_d['DU'] *0.02) + (df_ov_d['PH'] *0.14) + (df_ov_d['PA'] *0.12) + (df_ov_d['SC'] *0.16) + (df_ov_d['DF'] *0.15) + (df_ov_d['EX'] *0.03) + (df_ov_d['LD'] *0.04)
df_ov_f['OV'] = (df_ov_f['CK'] *0.07) + (df_ov_f['DI'] *0.02) + (df_ov_f['SK'] *0.10) + (df_ov_f['ST'] *0.05) + (df_ov_f['EN'] *0.09) + (df_ov_f['DU'] *0.02) + (df_ov_f['PH'] *0.14) + (df_ov_f['PA'] *0.14) + (df_ov_f['SC'] *0.19) + (df_ov_f['DF'] *0.11) + (df_ov_f['EX'] *0.03) + (df_ov_f['LD'] *0.03) + (df_ov_f['FO'] *0.01)

df_ov_rate = pd.concat([df_ov_d,df_ov_f])
df_ov_rate['Player'] = df_ov_rate['Player'].str.title()
df_ov_rate.drop_duplicates(subset='Player',inplace=True)
df_ov_rate.rename(columns={'height_in': 'Height'}, inplace=True)
df_ov_rate['League'] = 'NHL'

df_to_print = df_ov_rate.sort_values(by=['OV'],ascending=False)

df_to_print.to_csv("NHL_Skater_Ratings_2021-2022.csv",index=False,encoding='utf-8')

df_to_print

<ipython-input-95-f2e579a5ac1b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ov_d['OV'] = (df_ov_d['CK'] *0.08) + (df_ov_d['DI'] *0.02) + (df_ov_d['SK'] *0.10) + (df_ov_d['ST'] *0.05) + (df_ov_d['EN'] *0.09) + (df_ov_d['DU'] *0.02) + (df_ov_d['PH'] *0.14) + (df_ov_d['PA'] *0.12) + (df_ov_d['SC'] *0.16) + (df_ov_d['DF'] *0.15) + (df_ov_d['EX'] *0.03) + (df_ov_d['LD'] *0.04)
<ipython-input-95-f2e579a5ac1b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ov_f['OV'] = (df_ov_f['CK'] *0.07) + (df_ov

,Id,Player,Position,Birth Country,Rookie,Year,Month,Day,Weight,Height,...,FO,PA,SC,DF,PS,EX,LD,PO,OV,League
401,8478402,Connor Mcdavid,C,CAN,0,1997,01,13,193,73,...,85.090909,96.254346,85.377184,67.934333,79.380000,69.632595,80.313259,1,85.793700,NHL
439,8479318,Auston Matthews,C,USA,0,1997,09,17,205,75,...,90.181818,84.136791,97.718124,69.315223,74.249955,67.059328,66.158874,1,84.224485,NHL
335,8477493,Aleksander Barkov,C,FIN,0,1995,09,02,215,75,...,90.181818,85.727560,86.915914,73.717415,79.167780,73.138670,76.031514,1,83.529850,NHL
627,8477934,Leon Draisaitl,C,DEU,0,1995,10,27,208,74,...,88.090909,84.369637,89.612960,66.649462,65.362500,71.916369,66.644578,1,83.390930,NHL
474,8478864,Kirill Kaprizov,L,RUS,0,1997,04,26,202,70,...,54.750000,87.177024,85.869472,68.034344,68.357074,58.342387,58.034239,1,82.406533,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,8481552,Artemi Kniazev,D,RUS,1,2001,01,04,178,71,...,40.000000,59.000000,53.000000,67.133028,54.000000,54.000000,57.600000,1,59.525000,NHL
741,8473463,Leo Komarov,R,EST,0,1987,01,23,205,71,...,51.000000,59.000000,53.000000,59.945062,54.000000,69.761258,66.429067,1,58.981304,NHL
740,8477425,Miles Wood,L,USA,0,1995,09,13,195,74,...,51.000000,59.000000,53.000000,59.673596,54.000000,64.453896,64.971860,1,58.962029,NHL
739,8478029,Chase Deleo,C,USA,1,1995,10,25,185,69,...,50.000000,59.000000,53.000000,60.000000,54.000000,54.192995,57.619299,1,58.232954,NHL
